In [36]:
import nltk
import random
import json
from nltk.corpus import wordnet
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem import WordNetLemmatizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics import accuracy_score

In [37]:
intents = {
 "intents": [
    {
      "tag": "greeting",
      "patterns": ["Hi", "Hello", "Xin chào", "Chào buổi"],
      "responses": [
        "Xin chào!",
        "Vui mừng được gặp bạn!",
        "Chào, tôi có thể giúp gì cho bạn?"
      ],
      "context_set": ""
    },
       {
      "tag": "name_school",
      "patterns": ["Trường tên là gì?", "Tên đầy đủ của trường?", "Tên trường"],
      "responses": [
       "Trường Đại Học Công Nghệ Thông Tin và Truyền thông Việt Hàn"
      ],
      "context_set": ""
    },
    {
      "tag": "farewell",
      "patterns": ["Tạm biệt", "Bye", "Hẹn gặp lại"],
      "responses": [
        "Tiếc quá, bạn đi rồi :(",
        "Tạm biệt!",
        "Hãy quay lại sớm nhé!"
      ],
      "context_set": ""
    },
    {
      "tag": "creator",
      "patterns": [
        "Ai đã tạo ra bạn?",
        "Người phát triển ra bạn là ai?",
        "Bạn được ai tạo ra?"
      ],
      "responses": ["Tôi được VKU tạo ra."],
      "context_set": ""
    },
    {
      "tag": "identity",
      "patterns": ["Tên bạn là gì?", "Tôi nên gọi bạn là gì?", "Bạn là ai?"],
      "responses": ["Bạn có thể gọi tôi là VKUer. Tôi là một Chatbot."],
      "context_set": ""
    },
    {
      "tag": "hours",
      "patterns": [
        "Trường mở cửa vào những giờ nào?",
        "Trường hoạt động vào những thời gian nào?",
        "Trường mở cửa từ mấy giờ đến mấy giờ?"
      ],
      "responses": [
        "Trường mở cửa từ 7 giờ sáng đến 5 giờ chiều, từ Thứ Hai đến Thứ Bảy."
      ],
      "context_set": ""
    },
    {
      "tag": "contact",
      "patterns": [
        "Làm thế nào để liên lạc với trường?",
        "Số điện thoại của trường là gì?",
        "Tôi có thể nhận được số liên lạc của trường không?"
      ],
      "responses": ["Bạn có thể liên lạc với trường theo số 123456789."],
      "context_set": ""
    },
    {
      "tag": "courses",
      "patterns": [
        "Trường có những ngành học nào?",
        "Bạn có thể cho tôi biết các ngành học có sẵn không?",
        "Trường có những ngành gì?"
      ],
      "responses": [
        "Trường cung cấp các khóa học về Công nghệ Thông tin, Kỹ thuật Máy tính,Truyền thông đa phương tiện, quản trị kinh doanh,…."
      ],
      "context_set": ""
    },
    {
      "tag": "fees",
      "patterns": [
        "Học phí của trường là bao nhiêu?",
        "Hãy cho tôi biết về học phí?",
        "Học phí ký túc xá là bao nhiêu?"
      ],
      "responses": [
        "Để biết thông tin chi tiết về học phí, vui lòng truy cập trang web của trường."
      ],
      "context_set": ""
    },
    {
      "tag": "location",
      "patterns": [
        "Trường nằm ở đâu?",
        "Địa chỉ của trường là gì?",
        "Làm thế nào để tới được trường?"
      ],
      "responses": [
        "Trường nằm tại 470 Trần Đại Nghĩa, Ngũ Hành Sơn, Đà Nẵng. Bạn có thể tìm vị trí trên Google Maps."
      ],
      "context_set": ""
    },
    {
      "tag": "hostel",
      "patterns": [
        "Trường có cung cấp ký túc xá không?",
        "Ký túc xá nằm ở đâu?",
        "Học phí ký túc xá là bao nhiêu?"
      ],
      "responses": [
        "Trường có kí túc xá nằm ngay cạnh trường, Để biết thông tin chi tiết về ký túc xá, vui lòng truy cập trang web của trường."
      ],
      "context_set": ""
    },
    {
      "tag": "events",
      "patterns": [
        "Trường có tổ chức những sự kiện gì?",
        "Có những sự kiện sắp diễn ra không?",
        "Hãy cho tôi biết về các sự kiện của trường."
      ],
      "responses": [
        "Trường thường xuyên tổ chức các sự kiện như Robocar, Hội trại truyền thống, giao lưu văn hóa với nhiều nước, Để biết thông tin về các sự kiện, vui lòng truy cập trang web của trường."
      ],
      "context_set": ""
    },
    {
      "tag": "admission",
      "patterns": [
        "Quy trình nhập học là gì?",
        "Làm thế nào để vào học tại trường?",
        "Hãy cho tôi biết về tiêu chí nhập học."
      ],
      "responses": [
        "Để biết thông tin chi tiết về nhập học, vui lòng truy cập trang web của trường."
      ],
      "context_set": ""
    },
    {
      "tag": "library",
      "patterns": [
        "Trường có thư viện không?",
        "Thư viện nằm ở đâu?",
        "Thư viện mở cửa từ mấy giờ đến mấy giờ?"
      ],
      "responses": [
        "Có, trường có một thư viện. Thư viện nằm ở ngay cạnh trường. Thời gian mở cửa từ 9 giờ sáng đến 5 giờ chiều, Thứ Hai đến Thứ Sáu."
      ],
      "context_set": ""
    },
    {
      "tag": "facilities",
      "patterns": [
        "Trường có những cơ sở vật chất nào?",
        "Hãy cho tôi biết về các cơ sở vật chất của trường.",
        "Trường có khu thể thao không?"
      ],
      "responses": [
        "Trường cung cấp các cơ sở vật chất như khu thể thao, phòng thí nghiệm, căng tin và thư viện."
      ],
      "context_set": ""
    },
    {
      "tag": "placement",
      "patterns": [
        "Việc làm sau khi tốt nghiệp ở trường này thế nào?",
        "Hãy cho tôi biết về việc làm sau khi tốt nghiệp của trường.",
        "Thành tích việc làm sau khi tốt nghiệp của trường thế nào?"
      ],
      "responses": [
        "Trường có thành tích tốt về việc làm sau khi tốt nghiệp. Nhiều công ty uy tín đến trường chúng tôi để tuyển dụng."
      ],
      "context_set": ""
    },
    {
      "tag": "school_choice",
      "patterns": [
        "Tại sao bạn chọn trường này?",
        "Điều gì khiến bạn quyết định học ở trường này?"
      ],
      "responses": [
        "Tôi chọn trường này vì danh tiếng học thuật mạnh mẽ, giảng viên xuất sắc và các khóa học đa dạng."
      ],
      "context_set": ""
    },
    {
      "tag": "happiness",
      "patterns": [
        "Bạn có vui vẻ ở đây không?",
        "Bạn có thích học ở trường này không?"
      ],
      "responses": [
        "Là một AI, tôi không có cảm xúc, nhưng tôi ở đây để hỗ trợ và cung cấp thông tin về trường."
      ],
      "context_set": ""
    },
    {
      "tag": "strengths",
      "patterns": [
        "Điểm mạnh của trường bạn là gì?",
        "Điều gì làm cho trường bạn nổi bật?"
      ],
      "responses": [
        "Trường chúng tôi xuất sắc trong việc cung cấp giáo dục chất lượng, tạo ra môi trường học tập hỗ trợ và cung cấp nhiều hoạt động ngoại khóa đa dạng."
      ],
      "context_set": ""
    },
    {
      "tag": "complaints",
      "patterns": [
        "Phàn nàn lớn nhất của bạn về trường là gì?",
        "Bạn nghĩ trường cần cải thiện điều gì?"
      ],
      "responses": [
        "Mục đích của tôi là cung cấp thông tin, một số sinh viên có thể có ý kiến khác nhau. Luôn luôn tốt để đưa ra phản hồi nhằm cải thiện liên tục."
      ],
      "context_set": ""
    },
    {
      "tag": "improvements",
      "patterns": [
        "Những gì khác cần được cải thiện?",
        "Bạn nghĩ trường của bạn cần cải thiện ở những lĩnh vực nào?"
      ],
      "responses": [
        "Cải thiện liên tục là điều cần thiết cho bất kỳ tổ chức nào. Các lĩnh vực có thể xem xét cải thiện bao gồm cơ sở hạ tầng, các tiện nghi và dịch vụ hỗ trợ sinh viên."
      ],
      "context_set": ""
    },
    {
      "tag": "accessibility",
      "patterns": [
        "Quản trị viên, nhân viên đăng ký, nhân viên hỗ trợ tài chính, v.v. có dễ tiếp cận không?",
        "Sinh viên có thể dễ dàng liên lạc với nhân viên nhà trường khi cần không?"
      ],
      "responses": [
        "Ban quản lý trường luôn cố gắng duy trì khả năng tiếp cận cho sinh viên. Giờ làm việc và thông tin liên lạc của các nhân viên khác nhau đều có sẵn để hỗ trợ sinh viên."
      ],
      "context_set": ""
    },
    {
      "tag": "academics_majors",
      "patterns": [
        "Ngành học nào phổ biến?",
         "Ngành học được học nhiều nhất",
        "Những lĩnh vực học nào có nhiều sinh viên đăng ký?",
        "Ngành học nào được nhiều người lựa chọn nhất ở trường bạn?"
      ],
      "responses": [
        "Ngành học phổ biến tại trường chúng tôi bao gồm Khoa học Máy tính, Quản trị Kinh doanh, Tâm lý học, Kỹ thuật và Sinh học."
      ],
      "context_set": ""
    },
    {
      "tag": "academics_reputation",
      "patterns": [
        "Khoa hoặc chương trình nào có danh tiếng tốt nhất?",
        "Chương trình học nào được đánh giá cao?",
        "Có bộ môn nào nổi bật ở trường bạn không?"
      ],
      "responses": [
        "Trường chúng tôi có các khoa Khoa học Máy tính, Kỹ thuật và Quản trị Kinh doanh được đánh giá cao. Các chương trình này có danh tiếng mạnh mẽ về học thuật xuất sắc."
      ],
      "context_set": ""
    },
    {
      "tag": "favorite_class",
      "patterns": [
        "Lớp học yêu thích của bạn là gì?",
        "Bạn thích lớp nào nhất?",
        "Bạn có môn học yêu thích không?"
      ],
      "responses": [
        "Là một AI, tôi không có sở thích cá nhân. Tuy nhiên, sinh viên thường thích các lớp học phù hợp với sở thích và mục tiêu nghề nghiệp của họ."
      ],
      "context_set": ""
    },
    {
      "tag": "professors_teachers",
      "patterns": [
        "Giảng viên của bạn có phải là những người dạy tốt không?",
        "Chất lượng giảng dạy ở trường bạn thế nào?",
        "Trường có đội ngũ giảng viên giàu kinh nghiệm không?"
      ],
      "responses": [
        "Trường chúng tôi tự hào có đội ngũ giảng viên giàu kinh nghiệm và tận tâm, luôn cam kết cung cấp giáo dục chất lượng."
      ],
      "context_set": ""
    },
    {
      "tag": "professors_office_hours",
      "patterns": [
        "Giảng viên của bạn có giờ làm việc cố định không, và họ có gặp bạn ngoài giờ học không?",
        "Giảng viên có sẵn sàng gặp gỡ cá nhân không?",
        "Sinh viên có dễ dàng gặp Giảng viên không?"
      ],
      "responses": [
        "Hầu hết các giáo sư tại trường chúng tôi đều có giờ làm việc cố định và sẵn sàng gặp gỡ cá nhân để giải đáp thắc mắc của sinh viên và cung cấp thêm hướng dẫn."
      ],
      "context_set": ""
    },
    {
      "tag": "professors_vs_TAs",
      "patterns": [
        "Phần lớn các lớp học của bạn do giáo sư hay trợ giảng dạy?",
        "Ai là người dạy chính các lớp học?",
        "Bạn có nhiều lớp do trợ giảng dạy không?"
      ],
      "responses": [
        "Phần lớn các lớp học tại trường chúng tôi do các giáo sư giảng dạy. Tuy nhiên, trong một số trường hợp, trợ giảng có thể hỗ trợ giảng dạy hoặc tổ chức các buổi thảo luận."
      ],
      "context_set": ""
    },
    {
      "tag": "first_choice_classes",
      "patterns": [
        "Bạn có thể đăng ký được hầu hết các lớp học mà bạn mong muốn không?",
        "Sinh viên có thường xuyên được tham gia các lớp mà họ mong muốn không?",
        "Khả năng bạn được vào các lớp mà bạn mong muốn như thế nào?"
      ],
      "responses": [
        "Mặc dù sự sẵn có của khóa học có thể thay đổi, trường chúng tôi luôn cố gắng đáp ứng sở thích của sinh viên càng nhiều càng tốt. Đăng ký khóa học được thực hiện trên cơ sở ai đến trước được phục vụ trước."
      ],
      "context_set": ""
    },
    {
      "tag": "class_format",
      "patterns": [
        "Lớp học của bạn là dạng giảng bài hay thảo luận?",
        "Cấu trúc của các lớp học thế nào?",
        "Định dạng điển hình của một lớp học là gì?"
      ],
      "responses": [
        "Định dạng lớp học có thể thay đổi tùy theo môn học và phong cách giảng dạy của giảng viên. Một số lớp có thể là dạng giảng bài, trong khi các lớp khác có thể liên quan đến thảo luận và hoạt động nhóm."
      ],
      "context_set": ""
    },
    {
      "tag": "reading_writing",
      "patterns": [
        "Các khóa học của bạn yêu cầu bao nhiêu đọc và viết?",
        "Có nhiều bài tập liên quan đến đọc và viết không?",
        "Có chú trọng nhiều đến công việc viết không?"
      ],
      "responses": [
        "Lượng đọc và viết yêu cầu trong các khóa học có thể thay đổi. Nói chung, sinh viên có thể mong đợi sự kết hợp cân bằng giữa các bài đọc, dự án viết và các bài kiểm tra."
      ],
      "context_set": ""
    },
    {
      "tag": "collaborative_work",
      "patterns": [
        "Công việc hợp tác hoặc thuyết trình nhóm được yêu cầu thường xuyên như thế nào?",
        "Bạn có nhiều dự án nhóm không?",
        "Có chú trọng đến làm việc nhóm không?"
      ],
      "responses": [
        "Công việc hợp tác và thuyết trình nhóm là một phần của trải nghiệm học tập tại trường chúng tôi. Sinh viên thường tham gia vào các dự án nhóm để nâng cao kỹ năng làm việc nhóm và phát triển các kỹ năng quan trọng."
      ],
      "context_set": ""
    },
    {
      "tag": "academic_advising",
      "patterns": [
        "Bạn hài lòng với chương trình tư vấn học tập không?",
        "Chương trình tư vấn học tập có hữu ích không?",
        "Các cố vấn có cung cấp hướng dẫn về lựa chọn khóa học không?"
      ],
      "responses": [
        "Tư vấn học tập đóng vai trò quan trọng trong việc hướng dẫn hành trình học tập của sinh viên. Trường chúng tôi có các cố vấn tận tâm cung cấp sự trợ giúp về lựa chọn khóa học, lập kế hoạch bằng cấp và hỗ trợ học tập."
      ],
      "context_set": ""
    },
    {
      "tag": "tutoring_programs",
      "patterns": [
        "Sinh viên có sử dụng các chương trình dạy kèm hoặc trung tâm viết lách trong khuôn viên trường không?",
        "Có nguồn hỗ trợ học tập nào không?",
        "Làm thế nào để tôi có thể nhận trợ giúp dạy kèm?"
      ],
      "responses": [
        "Trường đại học của chúng tôi cung cấp các chương trình dạy kèm và trung tâm dạy Toeic trong khuôn viên để hỗ trợ sinh viên trong việc học tập. Các nguồn tài nguyên này có thể giúp sinh viên với sự hỗ trợ cụ thể về môn học và kỹ năng viết."
      ],
      "context_set": ""
    },
    {
      "tag": "class_size",
      "patterns": [
        "Lớp học có bao nhiêu sinh viên?",
        "Quy mô lớp học lớn hay nhỏ?",
        "Lớp học có quy mô nhỏ không?"
      ],
      "responses": [
        "Quy mô lớp học có thể thay đổi tùy theo khóa học và chương trình. Một số lớp có thể có số lượng sinh viên lớn hơn, trong khi các lớp học cấp cao hơn thường có quy mô nhỏ hơn để có sự chú ý cá nhân hơn."
      ],
      "context_set": ""
    },
    {
      "tag": "independent_study",
      "patterns": [
        "Ngành học của bạn có yêu cầu nghiên cứu độc lập hoặc dự án tốt nghiệp không?",
        "Có cơ hội nào cho nghiên cứu độc lập không?",
        "Ngành học của bạn có yêu cầu dự án kết thúc không?"
      ],
      "responses": [
        "Nhiều ngành học tại trường đại học của chúng tôi cung cấp cơ hội cho nghiên cứu độc lập hoặc dự án tốt nghiệp. Những trải nghiệm này cho phép sinh viên tìm hiểu sâu hơn về lĩnh vực quan tâm của họ và áp dụng kiến thức của mình."
      ],
      "context_set": ""
    },
    {
      "tag": "professor_research",
      "patterns": [
        "Giảng viên có sẵn sàng cho nghiên cứu với sinh viên không?",
        "Sinh viên có thể tham gia vào các dự án nghiên cứu với giảng viên không?",
        "Làm thế nào để tôi có thể tham gia vào nghiên cứu tại trường đại học của bạn?"
      ],
      "responses": [
        "Các giảng viên tại trường đại học của chúng tôi thường tham gia vào các dự án nghiên cứu, và sinh viên thường có cơ hội hợp tác trong các dự án này. Bạn có thể liên hệ với các giáo sư trong lĩnh vực quan tâm của mình để hỏi về các cơ hội nghiên cứu."
      ],
      "context_set": ""
    },
    {
      "tag": "study_abroad",
      "patterns": [
        "Du học có phổ biến không?",
        "Có các chương trình du học không?",
        "Sinh viên có thể du học một học kỳ không?"
      ],
      "responses": [
        "Du học là một lựa chọn phổ biến cho nhiều sinh viên tại trường đại học của chúng tôi. Chúng tôi cung cấp nhiều chương trình du học và cơ hội để trải nghiệm các nền văn hóa khác nhau và có được sự tiếp xúc quốc tế."
      ],
      "context_set": ""
    },
    {
      "tag": "first_year_experience",
      "patterns": [
        "Là sinh viên năm nhất tại đây như thế nào?",
        "Bạn có thể mô tả trải nghiệm năm nhất không?",
        "Tôi nên mong đợi gì khi là sinh viên năm nhất?"
      ],
      "responses": [
        "Là sinh viên năm nhất, bạn có thể mong đợi một môi trường hỗ trợ và chào đón. Các chương trình định hướng, tư vấn viên đồng trang lứa, và các dịch vụ hỗ trợ học tập đều có sẵn để giúp bạn chuyển tiếp suôn sẻ vào cuộc sống đại học."
      ],
      "context_set": ""
    },
    {
      "tag": "typical_day",
      "patterns": [
        "Một ngày điển hình như thế nào?",
        "Bạn có thể mô tả một ngày của sinh viên không?",
        "Lịch trình hàng ngày tại trường đại học của bạn như thế nào?"
      ],
      "responses": [
        "Một ngày điển hình của sinh viên bao gồm tham gia các lớp học, tham gia vào các buổi học nhóm, tham gia các hoạt động ngoại khóa và sử dụng các tài nguyên của khuôn viên trường. Lịch trình hàng ngày của mỗi sinh viên có thể khác nhau dựa trên lịch học và sở thích của họ."
      ],
      "context_set": ""
    },
    {
      "tag": "study_time",
      "patterns": [
        "Sinh viên dành bao nhiêu thời gian học mỗi tuần?",
        "Có yêu cầu học tập nhiều không?",
        "Thời gian học tập trung bình của sinh viên là bao nhiêu?"
      ],
      "responses": [
        "Thời gian học tập của sinh viên có thể thay đổi tùy theo khối lượng khóa học, thói quen học tập và sở thích cá nhân. Trung bình, sinh viên dành một số giờ mỗi tuần để học tập."
      ],
      "context_set": ""
    },
    {
      "tag": "non_class_activities",
      "patterns": [
        "Bạn làm gì khi không có lớp học?",
        "Sinh viên dành thời gian rảnh rỗi như thế nào?",
        "Có các hoạt động nào ngoài lớp học không?"
      ],
      "responses": [
        "Ngoài lớp học, sinh viên tham gia vào nhiều hoạt động như tham gia các câu lạc bộ và tổ chức, tham dự các sự kiện trong khuôn viên, khám phá cộng đồng xung quanh, và dành thời gian với bạn bè."
      ],
      "context_set": ""
    },
    {
      "tag": "weekend_activities",
      "patterns": [
        "Bạn làm gì vào cuối tuần?",
        "Có các hoạt động hoặc sự kiện vào cuối tuần không?",
        "Sinh viên dành cuối tuần như thế nào?"
      ],
      "responses": [
        "Vào cuối tuần, sinh viên thường tham gia vào các sự kiện trong khuôn viên, các hoạt động giải trí, các buổi tụ tập xã hội, và cũng có thể dành thời gian để thư giãn và theo đuổi sở thích cá nhân."
      ],
      "context_set": ""
    },
    {
      "tag": "social_scene",
      "patterns": [
        "Cuộc sống xã hội tại trường như thế nào?",
        "Bạn mô tả cuộc sống xã hội trong khuôn viên trường như thế nào?",
        "Có cơ hội để gặp gỡ những người mới không?"
      ],
      "responses": [
        "Cuộc sống xã hội trong khuôn viên trường rất sôi động và đa dạng. Có nhiều cơ hội để gặp gỡ những người mới, tham gia các câu lạc bộ và tổ chức, tham dự các sự kiện xã hội, và xây dựng các mối quan hệ bền vững."
      ],
      "context_set": ""
    },
    {
      "tag": "hometown_activities",
      "patterns": [
        "Có những hoạt động gì trong thị trấn nơi trường học của bạn?",
        "Có các hoạt động giải trí trong khu vực địa phương không?",
        "Các điểm tham quan phổ biến gần trường đại học là gì?"
      ],
      "responses": [
        "Thị trấn nơi trường học của chúng tôi cung cấp một loạt các hoạt động giải trí, sự kiện văn hóa, trung tâm mua sắm, nhà hàng và các điểm tham quan thiên nhiên. Bạn có thể khám phá các công viên địa phương, bảo tàng, nhà hát và thưởng thức ẩm thực địa phương."
      ],
      "context_set": ""
    },
    {
      "tag": "food",
      "patterns": [
        "Thức ăn như thế nào?",
        "Chất lượng thức ăn trong khuôn viên trường như thế nào?",
        "Có các lựa chọn ăn uống tốt không?"
      ],
      "responses": [
        "Trường đại học cung cấp nhiều lựa chọn ăn uống trong khuôn viên, bao gồm các nhà ăn, khu ẩm thực và nhà hàng đặc biệt. Chất lượng thức ăn thường khá tốt, với các lựa chọn phục vụ cho các sở thích ăn uống khác nhau."
      ],
      "context_set": ""
    },
    {
      "tag": "on_campus_housing",
      "patterns": [
        "Phần lớn sinh viên có sống trong khuôn viên trường không?",
        "Có lựa chọn nhà ở trong khuôn viên trường không?",
        "Tỷ lệ phần trăm sinh viên sống trong ký túc xá là bao nhiêu?"
      ],
      "responses": [
        "Mặc dù sự có sẵn của nhà ở trong khuôn viên trường có thể khác nhau, nhiều sinh viên chọn sống trong ký túc xá hoặc nhà ở do trường cung cấp. Cũng có các lựa chọn nhà ở ngoài khuôn viên trường cho sinh viên."
      ],
      "context_set": ""
    },
    {
      "tag": "dorm_quality",
      "patterns": [
        "Ký túc xá như thế nào?",
        "Ký túc xá có thoải mái không?",
        "Chất lượng nhà ở trong khuôn viên trường ra sao?"
      ],
      "responses": [
        "Chất lượng của các ký túc xá khác nhau tùy thuộc vào từng tòa nhà cụ thể. Nhìn chung, trường cố gắng cung cấp không gian sống thoải mái và tiện nghi để đáp ứng nhu cầu của sinh viên."
      ],
      "context_set": ""
    },
    {
      "tag": "popular_clubs",
      "patterns": [
        "Những câu lạc bộ và tổ chức sinh viên nào phổ biến?",
        "Có câu lạc bộ nào nổi bật trong khuôn viên trường không?",
        "Một số tổ chức sinh viên hoạt động sôi nổi nhất là gì?"
      ],
      "responses": [
        "Trường của chúng tôi có nhiều câu lạc bộ và tổ chức sinh viên phục vụ các sở thích đa dạng. Những câu lạc bộ phổ biến bao gồm các tổ chức học thuật, câu lạc bộ văn hóa, đội thể thao, nhóm phục vụ cộng đồng và nhiều hơn nữa."
      ],
      "context_set": ""
    },
    {
      "tag": "fraternities_sororities",
      "patterns": [
        "Nhiều sinh viên có thuộc các hội nam sinh hoặc nữ sinh không?",
        "Có nhiều tổ chức Greek trong khuôn viên trường không?",
        "Đời sống Greek tại trường bạn như thế nào?"
      ],
      "responses": [
        "Mặc dù sự hiện diện và quan trọng của các hội nam sinh và nữ sinh có thể khác nhau giữa các trường, trường của chúng tôi có cộng đồng Greek. Việc tham gia vào các tổ chức này là lựa chọn cá nhân của mỗi sinh viên."
      ],
      "context_set": ""
    },
    {
      "tag": "favorite_place_on_campus",
      "patterns": [
        "Nơi yêu thích của bạn trong khuôn viên trường là gì?",
        "Bạn thích địa điểm nào nhất?",
        "Sinh viên thích tụ tập ở đâu?"
      ],
      "responses": [
        "Là một AI, tôi không có sở thích cá nhân. Tuy nhiên, nhiều sinh viên thích dành thời gian ở các khu vực chung, thư viện, trung tâm sinh viên, không gian ngoài trời và căn tin của trường."
      ],
      "context_set": ""
    },
    {
      "tag": "study_locations",
      "patterns": [
        "Bạn thích học ở đâu?",
        "Một số địa điểm học tập tốt là gì?",
        "Có nơi yên tĩnh để học trong khuôn viên trường không?"
      ],
      "responses": [
        "Có nhiều địa điểm học tập trong khuôn viên trường, bao gồm thư viện, phòng chờ học tập, quán cà phê và khu vực ngồi ngoài trời. Quan trọng là tìm một địa điểm phù hợp với sở thích học tập của bạn và giúp bạn tập trung."
      ],
      "context_set": ""
    },
    {
      "tag": "campus_facilities",
      "patterns": [
        "Các cơ sở vật chất như thế nào (phòng thí nghiệm khoa học, thư viện, nhà hát, phòng gym, v.v.)?",
        "Các cơ sở vật chất trong khuôn viên trường có được bảo trì tốt không?",
        "Trường có các cơ sở vật chất hiện đại không?"
      ],
      "responses": [
        "Trường của chúng tôi tự hào duy trì các cơ sở vật chất được trang bị tốt để hỗ trợ các hoạt động học tập và ngoại khóa của sinh viên. Điều này bao gồm các phòng thí nghiệm khoa học hiện đại, thư viện với nhiều tài nguyên, nhà hát và phòng gym được trang bị tốt."
      ],
      "context_set": ""
    },
    {
      "tag": "computer_labs",
      "patterns": [
        "Có đủ phòng máy tính không?",
        "Các phòng máy tính có dễ tiếp cận không?",
        "Có thiếu hụt các cơ sở phòng máy tính không?"
      ],
      "responses": [
        "Trường của chúng tôi cung cấp một số lượng lớn các phòng máy tính trên toàn khuôn viên trường. Các phòng này dễ tiếp cận với sinh viên và được trang bị công nghệ và phần mềm cần thiết cho công việc học tập."
      ],
      "context_set": ""
    },
    {
      "tag": "campus_wifi",
      "patterns": [
        "WiFi trong khuôn viên trường như thế nào?",
        "WiFi trong khuôn viên trường có ổn định không?",
        "Có vấn đề gì về kết nối WiFi không?"
      ],
      "responses": [
        "Mạng WiFi trong khuôn viên trường được thiết kế để cung cấp kết nối internet ổn định cho sinh viên, giảng viên và nhân viên. Tuy nhiên, đôi khi có thể xảy ra vấn đề kết nối, nhưng thường được bộ phận IT giải quyết nhanh chóng."
      ],
      "context_set": ""
    },
    {
      "tag": "campus_navigation",
      "patterns": [
        "Có dễ dàng để đi lại trong khuôn viên trường không?",
        "Các tòa nhà trong khuôn viên trường có dễ dàng điều hướng không?",
        "Bố trí khuôn viên trường có thân thiện với sinh viên không?"
      ],
      "responses": [
        "Khuôn viên trường được thiết kế để thân thiện với sinh viên, và các tòa nhà trong khuôn viên thường được đánh dấu rõ ràng và dễ dàng điều hướng. Bản đồ khuôn viên, biển chỉ dẫn và tài nguyên số có sẵn để giúp sinh viên tìm đường."
      ],
      "context_set": ""
    },
    {
      "tag": "sports_popularity",
      "patterns": [
        "Thể thao có phổ biến không?",
        "Nhiều sinh viên có tham gia các hoạt động thể thao không?",
        "Văn hóa thể thao tại trường bạn như thế nào?"
      ],
      "responses": [
        "Thể thao rất phổ biến tại trường của chúng tôi, và nhiều sinh viên tham gia vào các hoạt động thể thao khác nhau. Chúng tôi có nhiều đội thể thao, thể thao nội bộ và chương trình thể dục để phục vụ các sở thích và mức độ kỹ năng khác nhau."
      ],
      "context_set": ""
    },
    {
      "tag": "school_spirit",
      "patterns": [
        "Có nhiều tinh thần trường không?",
        "Sinh viên có thể hiện niềm tự hào về trường mạnh mẽ không?",
        "Có các sự kiện hoặc truyền thống thể hiện tinh thần trường không?"
      ],
      "responses": [
        "Trường của chúng tôi có tinh thần trường sôi động, và sinh viên thường thể hiện niềm tự hào mạnh mẽ về trường của mình. Có các sự kiện sôi động, truyền thống và cơ hội để tham gia cùng cộng đồng trường và thể hiện tinh thần trường."
      ],
      "context_set": ""
    },
    {
      "tag": "fellow_students",
      "patterns": [
        "Bạn sẽ mô tả các sinh viên khác như thế nào?",
        "Sinh viên như thế nào?",
        "Có sự gắn kết giữa các sinh viên không?"
      ],
      "responses": [
        "Đội ngũ sinh viên đa dạng và bao gồm những cá nhân từ nhiều hoàn cảnh, sở thích và trải nghiệm khác nhau. Sinh viên thường nuôi dưỡng một tinh thần cộng đồng và hỗ trợ sự phát triển học thuật và cá nhân của nhau."
      ],
      "context_set": ""
    },
    {
      "tag": "friendly_students",
      "patterns": [
        "Sinh viên ở đây có thân thiện không?",
        "Có dễ dàng để kết bạn không?",
        "Sinh viên có tương tác với nhau không?"
      ],
      "responses": [
        "Có, sinh viên ở đây thường rất thân thiện và chào đón. Môi trường trường học cung cấp nhiều cơ hội để gặp gỡ người mới, kết bạn và hình thành các mối quan hệ ý nghĩa cả trong và ngoài lớp học."
      ],
      "context_set": ""
    },
    {
      "tag": "student_body_unique",
      "patterns": [
        "Điều gì làm cho đội ngũ sinh viên trở nên đặc biệt?",
        "Có điều gì đặc biệt về đội ngũ sinh viên không?",
        "Điều gì làm cho sinh viên tại trường của bạn khác biệt?"
      ],
      "responses": [
        "Đội ngũ sinh viên tại trường của chúng tôi đặc biệt ở sự đa dạng về hoàn cảnh, quan điểm, tài năng và đam mê. Sự đa dạng này tạo ra một môi trường học tập phong phú và khuyến khích sinh viên chấp nhận các nền văn hóa và quan điểm khác nhau."
      ],
      "context_set": ""
    },
    {
      "tag": "campus_diversity",
      "patterns": [
        "Có sự đa dạng trong khuôn viên trường không?",
        "Có sinh viên từ các nền văn hóa khác nhau không?",
        "Trường có khuyến khích sự đa dạng và hòa nhập không?"
      ],
      "responses": [
        "Có, trường chúng tôi coi trọng sự đa dạng và nỗ lực tạo ra một cộng đồng khuôn viên bao gồm. Sinh viên từ các nền văn hóa, sắc tộc và quốc gia khác nhau đóng góp vào sự đa dạng phong phú của cộng đồng trường."
      ],
      "context_set": ""
    },
    {
      "tag": "international_students",
      "patterns": [
        "Có nhiều sinh viên từ các quốc gia khác không?",
        "Trường có một số lượng lớn sinh viên quốc tế không?",
        "Sự đa dạng quốc tế tại trường của bạn như thế nào?"
      ],
      "responses": [
        "Trường chúng tôi chào đón sinh viên từ khắp nơi trên thế giới và có một cộng đồng sinh viên quốc tế đa dạng. Sinh viên quốc tế đóng góp vào sự trao đổi văn hóa và các góc nhìn toàn cầu trong cộng đồng trường."
      ],
      "context_set": ""
    },
    {
      "tag": "interactions_among_students",
      "patterns": [
        "Sinh viên từ các chủng tộc và tầng lớp khác nhau có tương tác dễ dàng không?",
        "Có sự tương tác và giao lưu giữa các nhóm đa dạng không?",
        "Sinh viên có đón nhận sự đa dạng và tham gia vào các giao tiếp xuyên văn hóa không?"
      ],
      "responses": [
        "Sinh viên tại trường chúng tôi có cơ hội tương tác và giao lưu với bạn bè từ các chủng tộc, tầng lớp và nền văn hóa khác nhau. Trường khuyến khích một môi trường chào đón và bao gồm, tạo điều kiện cho các giao tiếp xuyên văn hóa và sự hiểu biết."
      ],
      "context_set": ""
    },
    {
      "tag": "student_cliques",
      "patterns": [
        "Sinh viên có chia bè kết phái không?",
        "Có sự hiện diện của các nhóm riêng biệt trong khuôn viên trường không?",
        "Sinh viên có chia thành các nhóm độc quyền không?"
      ],
      "responses": [
        "Mặc dù một số nhóm xã hội có thể tự nhiên hình thành dựa trên sở thích và liên kết chung, trường chúng tôi khuyến khích sự bao gồm và thúc đẩy một môi trường nơi sinh viên từ các nền tảng khác nhau có thể giao lưu và tạo kết nối ngoài các nhóm riêng."
      ],
      "context_set": ""
    },
    {
      "tag": "internship_opportunities",
      "patterns": [
        "Có các cơ hội thực tập không? Làm thế nào để tìm chúng?",
        "Có những cơ hội thực tập nào?",
        "Trường hỗ trợ sinh viên như thế nào trong việc tìm kiếm thực tập?"
      ],
      "responses": [
        "Có các cơ hội thực tập dành cho sinh viên, và trường thường cung cấp các tài nguyên và hỗ trợ để giúp sinh viên tìm và đảm bảo thực tập. Văn phòng dịch vụ nghề nghiệp của trường và các cố vấn giảng viên có thể cung cấp hướng dẫn trong việc khám phá và nộp đơn cho thực tập."
      ],
      "context_set": ""
    },
    {
      "tag": "career_services",
      "patterns": [
        "Dịch vụ nghề nghiệp có hữu ích không?",
        "Trung tâm nghề nghiệp cung cấp những dịch vụ gì?",
        "Sự hướng dẫn nghề nghiệp của trường có hiệu quả không?"
      ],
      "responses": [
        "Dịch vụ nghề nghiệp tại trường chúng tôi cung cấp nhiều tài nguyên và hỗ trợ để giúp sinh viên khám phá nghề nghiệp, xây dựng sơ yếu lý lịch, chiến lược tìm việc, chuẩn bị phỏng vấn và nhiều hơn nữa. Nhiều sinh viên thấy rằng sự hướng dẫn nghề nghiệp rất hữu ích cho sự phát triển chuyên môn của họ."
      ],
      "context_set": ""
    },
    {
      "tag": "mentorship_program",
      "patterns": [
        "Bạn có người cố vấn không?",
        "Có chương trình cố vấn không?",
        "Làm thế nào sinh viên có thể kết nối với các cố vấn?"
      ],
      "responses": [
        "Trường chúng tôi có thể có chương trình cố vấn hoặc các cơ hội để sinh viên kết nối với các cố vấn. Các chương trình này tạo điều kiện cho việc hướng dẫn và hỗ trợ từ những người có kinh nghiệm, cung cấp những kiến thức và lời khuyên quý báu về con đường học tập và nghề nghiệp."
      ],
      "context_set": ""
    },
    {
      "tag": "co-op_program",
      "patterns": [
        "Trường của bạn có chương trình co-op không?",
        "Có các cơ hội giáo dục hợp tác không?",
        "Trường hỗ trợ sinh viên như thế nào trong các trải nghiệm co-op?"
      ],
      "responses": [
        "Một số trường cung cấp các chương trình co-op, nơi sinh viên có thể có kinh nghiệm làm việc thực tế liên quan đến lĩnh vực học tập của họ trong khi kiếm tín chỉ học tập. Nếu trường chúng tôi có chương trình co-op, sinh viên có thể khám phá các cơ hội như vậy với sự hỗ trợ từ văn phòng dịch vụ nghề nghiệp hoặc giáo dục hợp tác của trường."
      ],
      "context_set": ""
    },
    {
      "tag": "leadership_opportunities",
      "patterns": [
        "Có những lựa chọn nào để phát triển kỹ năng lãnh đạo trong khuôn viên trường?",
        "Có các chương trình hoặc vị trí lãnh đạo không?",
        "Sinh viên có thể phát triển kỹ năng lãnh đạo như thế nào?"
      ],
      "responses": [
        "Trường chúng tôi cung cấp nhiều cơ hội để sinh viên phát triển kỹ năng lãnh đạo. Sinh viên có thể tham gia vào chính phủ sinh viên, các câu lạc bộ hoặc tổ chức cung cấp các vị trí lãnh đạo, tham gia các hội thảo hoặc chương trình lãnh đạo, và tham gia vào các hoạt động phục vụ cộng đồng để phát triển phẩm chất lãnh đạo."
      ],
      "context_set": ""
    },
    {
      "tag": "service_learning",
      "patterns": [
        "Có những lựa chọn nào cho việc học qua phục vụ cộng đồng?",
        "Có các chương trình phục vụ cộng đồng không?",
        "Sinh viên có thể tham gia vào phục vụ cộng đồng như thế nào?"
      ],
      "responses": [
        "Trường chúng tôi thường khuyến khích học qua phục vụ và tham gia cộng đồng. Sinh viên có thể tham gia vào các chương trình phục vụ cộng đồng, tình nguyện với các tổ chức địa phương, tham gia các khóa học học qua phục vụ, hoặc tham gia vào các sáng kiến trong khuôn viên trường nhằm tạo tác động tích cực trong cộng đồng."
      ],
      "context_set": ""
    },
    {
      "tag": "employer_recruitment",
      "patterns": [
        "Các nhà tuyển dụng có tuyển sinh viên trong khuôn viên trường không?",
        "Có các hội chợ nghề nghiệp hoặc hội chợ việc làm không?",
        "Trường kết nối sinh viên với các nhà tuyển dụng tiềm năng như thế nào?"
      ],
      "responses": [
        "Việc tuyển dụng nhà tuyển dụng là một hoạt động phổ biến tại nhiều trường đại học. Các hội chợ nghề nghiệp, hội chợ việc làm và các sự kiện kết nối được tổ chức để kết nối sinh viên với các nhà tuyển dụng tiềm năng. Văn phòng dịch vụ nghề nghiệp của trường thường đóng vai trò trong việc tạo điều kiện cho các kết nối này."
      ],
      "context_set": ""
    },
    {
      "tag": "summer_jobs",
      "patterns": [
        "Tìm việc mùa hè và các loại công việc khác qua trường có dễ không?",
        "Có các tài nguyên để tìm kiếm việc làm mùa hè không?",
        "Trường có hỗ trợ cơ hội việc làm mùa hè không?"
      ],
      "responses": [
        "Trường chúng tôi có thể cung cấp các tài nguyên và hỗ trợ để giúp sinh viên tìm việc làm mùa hè và các loại công việc khác. Văn phòng dịch vụ nghề nghiệp, các bảng tin việc làm trực tuyến và các sự kiện kết nối có thể rất có ích trong việc khám phá và đảm bảo cơ hội việc làm mùa hè."
      ],
      "context_set": ""
    },
    {
      "tag": "notable_graduates",
      "patterns": [
        "Có những cựu sinh viên nổi tiếng nào từ trường của bạn?",
        "Bạn có thể đề cập đến bất kỳ cựu sinh viên nổi tiếng nào không?",
        "Những cá nhân thành công nào đã học tại trường của bạn?"
      ],
      "responses": [
        "Trường của chúng tôi có một danh sách các cựu sinh viên nổi tiếng và cựu sinh viên thành công đáng chú ý trong nhiều lĩnh vực khác nhau. Một số có thể bao gồm các chuyên gia nổi tiếng, doanh nhân, nghệ sĩ, nhà khoa học và nhà lãnh đạo đã có những đóng góp đáng kể cho lĩnh vực của họ."
      ],
      "context_set": ""
    },
    {
      "tag": "alumni_association",
      "patterns": [
        "Hội cựu sinh viên của trường bạn có phổ biến trên khuôn viên không?",
        "Liên kết cựu sinh viên của trường có hoạt động không?",
        "Có cơ hội kết nối với cựu sinh viên không?"
      ],
      "responses": [
        "Nhiều trường đại học có các hiệp hội cựu sinh viên hoạt động cung cấp cơ hội mạng lưới, chương trình hướng dẫn và hỗ trợ nghề nghiệp cho sinh viên hiện tại. Tính phổ biến và sự tham gia của hiệp hội cựu sinh viên có thể thay đổi, nhưng thường có nỗ lực để kết nối sinh viên với cựu sinh viên."
      ],
      "context_set": ""
    },
    {
      "tag": "hands-on_experiences",
      "patterns": [
        "Bạn đã có những trải nghiệm thực tế nào trong và ngoài lớp học?",
        "Có cơ hội học hỏi thực hành không?",
        "Trường đại học của bạn nhấn mạnh việc học thông qua trải nghiệm như thế nào?"
      ],
      "responses": [
        "Các trường đại học thường cung cấp cơ hội trải nghiệm thực tế cả trong và ngoài lớp học. Những điều này có thể bao gồm thực tập, dự án nghiên cứu, công việc trên thực địa, học tình nguyện, chương trình du học và các dự án hợp tác giúp tăng cường quá trình học tập và cung cấp ứng dụng thực tế của kiến thức."
      ],
      "context_set": ""
    },
    {
      "tag": "special_features",
      "patterns": [
        "Điều gì làm cho trường đại học này đặc biệt?",
        "Dịch vụ gì làm cho trường của bạn nổi bật hơn so với các trường khác?",
        "Trường đại học của bạn cung cấp các chương trình hoặc cơ hội độc đáo nào?",
        "Hãy kể về các đặc điểm độc đáo của trường của bạn.",
        "Điểm mạnh hoặc điểm bán hàng độc đáo của trường là gì?"
      ],
      "responses": [
        "Trường của chúng tôi nổi tiếng với cam kết mạnh mẽ với việc học thông qua trải nghiệm thực hành, các chương trình du học phong phú và một cộng đồng sinh viên sôi động. Chúng tôi cũng tự hào về cơ sở nghiên cứu tiên tiến và một đội ngũ giáo viên tận tâm với sự thành công của sinh viên."
      ],
      "context_set": ""
    },
    {
      "tag": "college_pride",
      "patterns": [
        "Trường này tự hào về điều gì?",
        "Những thành tựu nổi bật nào của trường đại học được biết đến?",
        "Những gì khiến sinh viên tự hào khi theo học trường này?",
        "Hãy kể về những thành tựu đáng chú ý của trường.",
        "Trường đại học này thể hiện những thành tựu của mình như thế nào?"
      ],
      "responses": [
        "Trường của chúng tôi tự hào về cộng đồng đa dạng và bao gồm, các chương trình học cao cấp chất lượng và những cựu sinh viên thành công đã có đóng góp đáng kể trong nhiều lĩnh vực. Chúng tôi cũng tổ chức các mạng lưới cựu sinh viên mạnh mẽ và tác động tích cực của họ đối với xã hội."
      ],
      "context_set": ""
    },
    {
      "tag": "undergraduate_students",
      "patterns": [
        "Có bao nhiêu sinh viên đại học theo học tại trường này?",
        "Kích thước của sinh viên đại học là bao nhiêu?",
        "Bạn có thể cho biết tổng số sinh viên đại học?",
        "Kích thước trung bình của lớp học đại học là bao nhiêu?",
        "Có bao nhiêu sinh viên trong lớp sinh viên năm nhất?"
      ],
      "responses": [
        "Trường của chúng tôi có khoảng [X] sinh viên đại học. Kích thước trung bình của lớp học đại học thường là khoảng [Y] sinh viên, đảm bảo một trải nghiệm học tập cá nhân hóa và hấp dẫn cho sinh viên của chúng tôi."
      ],
      "context_set": ""
    },
    {
      "tag": "graduate_students",
      "patterns": [
        "Có bao nhiêu sinh viên sau đại học tham gia?",
        "Kích thước của dân số sinh viên sau đại học là bao nhiêu?",
        "Bạn có thể cho biết tổng số sinh viên sau đại học?",
        "Trường đại học cung cấp những chương trình sau đại học nào?",
        "Tỷ lệ sinh viên tham gia chương trình sau đại học là bao nhiêu phần trăm?"
      ],
      "responses": [
        "Trường của chúng tôi có một dân số sinh viên sau đại học đa dạng với khoảng [X] sinh viên. Chúng tôi cung cấp một loạt các chương trình sau đại học phục vụ cho các quan tâm học thuật và chuyên nghiệp khác nhau."
      ],
      "context_set": ""
    },
    {
      "tag": "campus_diversity",
      "patterns": [
        "Khuôn viên của trường này có đa dạng không?",
        "Phân loại dân số học sinh của khuôn viên ra sao?",
        "Trường đại học có dân số sinh viên đa dạng không?",
        "Hãy nói về cam kết của khuôn viên với đa dạng và sự bao gồm.",
        "Có bất kỳ sự kiện hoặc sáng kiến liên quan đến đa dạng nào trên khuôn viên không?"
      ],
      "responses": [
        "Trường của chúng tôi cam kết thúc đẩy đa dạng và sự bao gồm. Chúng tôi có một cộng đồng sinh viên đa dạng đại diện cho các nền văn hóa, dân tộc và danh tính khác nhau, làm phong phú cộng đồng khuôn viên của chúng tôi và nuôi dưỡng sự hiểu biết qua văn hóa."
      ],
      "context_set": ""
    },
    {
      "tag": "financial_stability",
      "patterns": [
        "Trường này tài chính ổn định đến mức nào?",
        "Trường đại học có ổn định về mặt tài chính không?",
        "Trường đại học có đối mặt với bất kỳ thách thức tài chính nào không?",
        "Kích thước quỹ đầu tư của trường là bao nhiêu?",
        "Trường đại học làm thế nào để quản lý nguồn lực tài chính của mình?"
      ],
      "responses": [
        "Trường của chúng tôi duy trì sự ổn định tài chính thông qua quản lý ngân sách cẩn thận, thực hành tài chính cẩn trọng và một quỹ đầu tư mạnh mẽ. Chúng tôi cam kết cung cấp cho sinh viên một nền giáo dục chất lượng cao và đảm bảo tính bền vững lâu dài."
      ],
      "context_set": ""
    },
    {
      "tag": "bond_rating",
      "patterns": [
        "Điểm danh giá của trường là bao nhiêu?",
        "Điểm danh giá trái phiếu của trường làm thế nào ảnh hưởng đến tình trạng tài chính của nó?",
        "Điểm danh giá tín dụng của trường là bao nhiêu?",
        "Bạn có thể cung cấp thông tin về điểm danh giá tài chính của trường không?",
        "Trường đại học sử dụng điểm danh giá trái phiếu của mình để đảm bảo nguồn quỹ?"
      ],
      "responses": [
        "Điểm danh giá trái phiếu của trường chúng tôi là [X], cho thấy tính khả năng thanh toán nợ và sự ổn định tài chính mạnh mẽ. Một điểm danh giá trái phiếu cao hơn cho phép trường đại học đảm bảo nguồn quỹ với lãi suất thuận lợi hơn, hỗ trợ các dự án và sáng kiến trên khuôn viên khác nhau."
      ],
      "context_set": ""
    },
    {
      "tag": "average_class_size_intro",
      "patterns": [
        "Trung bình kích thước lớp học của các khóa học giới thiệu là bao nhiêu?",
        "Bao nhiêu sinh viên thường có trong các lớp học giới thiệu?",
        "Các khóa học giới thiệu có lớn không?",
        "Có nhiều sinh viên tham gia các khóa học giới thiệu không?",
        "Sinh viên có thể mong đợi nhận được sự chú ý cá nhân bao nhiêu trong các khóa học giới thiệu?"
      ],
      "responses": [
        "Các khóa học giới thiệu tại trường chúng tôi thường có kích thước lớp trung bình là [Y] sinh viên. Đội ngũ giáo viên của chúng tôi cam kết cung cấp sự chú ý và hỗ trợ cá nhân cho sinh viên, đảm bảo một trải nghiệm học tập có giá trị."
      ],
      "context_set": ""
    },
    {
      "tag": "average_class_size_upper_division",
      "patterns": [
        "Kích thước trung bình của các lớp học phần cao hơn là bao nhiêu?",
        "Các lớp học phần cao hơn có lớn hay nhỏ?",
        "Các lớp học phần cao hơn có nhiều sinh viên không?",
        "Sinh viên có thể mong đợi nhận được sự chú ý cá nhân bao nhiêu trong các lớp học phần cao hơn?",
        "Các lớp học phần cao hơn có môi trường giao tiếp gần gũi hơn so với các lớp học giới thiệu không?"
      ],
      "responses": [
        "Các lớp học phần cao hơn tại trường chúng tôi thường có kích thước lớp trung bình là [Y] sinh viên. Kích thước lớp nhỏ hơn cho phép các tương tác cá nhân hóa hơn giữa sinh viên và giáo viên, tạo điều kiện cho một môi trường học tập hợp tác."
      ],
      "context_set": ""
    },
    {
      "tag": "teaching_assistants",
      "patterns": [
        "Teaching assistants được sử dụng thường xuyên như thế nào?",
        "Teaching assistants có tham gia giảng dạy không?",
        "Teaching assistants có dẫn dắt thảo luận hoặc thí nghiệm không?",
        "Teaching assistants dễ tiếp cận với sinh viên như thế nào?",
        "Vai trò của teaching assistants trong hỗ trợ sinh viên là gì?"
      ],
      "responses": [
        "Các teaching assistant được sử dụng trong một số khóa học tại trường của chúng tôi để hỗ trợ giảng viên và nâng cao trải nghiệm học tập. Họ có thể hỗ trợ trong việc dẫn dắt thảo luận, thí nghiệm hoặc chấm điểm bài tập. Teaching assistants thường dễ tiếp cận với sinh viên trong giờ làm việc để cung cấp sự trợ giúp bổ sung."
      ],
      "context_set": ""
    },
    {
      "tag": "course_enrollment_ease",
      "patterns": [
        "Việc đăng ký các khóa học bạn cần có dễ hay khó?",
        "Quy trình đăng ký khóa học có phức tạp không?",
        "Sinh viên có thể đăng ký vào các khóa học ưa thích của họ không?",
        "Sinh viên có gặp bất kỳ khó khăn nào trong quá trình đăng ký khóa học không?",
        "Trường đại học làm thế nào để đảm bảo sinh viên có thể đăng ký vào các khóa học bắt buộc?"
      ],
      "responses": [
        "Việc đăng ký khóa học tại trường của chúng tôi thường là một quy trình đơn giản, và những nỗ lực được thực hiện để thỏa mãn sở thích của sinh viên. Cố vấn học thuật sẵn có để hỗ trợ sinh viên trong việc lựa chọn các khóa học phù hợp và đảm bảo họ có thể đăng ký vào các lớp học cần thiết để tiến bộ trong hành trình học tập của mình."
      ],
      "context_set": ""
    },
    {
      "tag": "interaction_with_professors",
      "patterns": [
        "Mức độ tương tác, nếu có, mà sinh viên thông thường sẽ có với giáo sư đã có chức vụ?",
        "Liệu việc tương tác với giáo sư đã có chức vụ có bị hạn chế trong phòng học?",
        "Cơ hội nào dành cho sinh viên để tương tác với giáo sư ngoài lớp học?",
        "Giáo sư có tổ chức giờ làm việc đều đặn cho sinh viên không?",
        "Có cơ hội để hợp tác với giáo sư trong các dự án nghiên cứu không?"
      ],
      "responses": [
        "Trường của chúng tôi thúc đẩy các tương tác ý nghĩa giữa sinh viên và giáo sư đã có chức vụ. Trong khi một số bài giảng có thể có kích thước lớp lớn, các giáo sư thường tổ chức giờ làm việc đều đặn để cung cấp sự hỗ trợ và hướng dẫn cá nhân. Sinh viên cũng có cơ hội hợp tác với giáo sư trong các dự án nghiên cứu hoặc tham gia các câu lạc bộ học thuật do giáo viên dẫn dắt."
      ],
      "context_set": ""
    },
    {
      "tag": "typical_class_sizes_major",
      "patterns": [
        "Kích thước lớp thông thường của chuyên ngành tôi là bao nhiêu?",
        "Các lớp học trong chuyên ngành tôi thường có kích thước lớn hay nhỏ?",
        "Sinh viên có thể mong đợi nhận được sự chú ý cá nhân trong các khóa học liên quan đến chuyên ngành không?",
        "Các lớp học cụ thể cho mỗi chuyên ngành có ít gần gũi hơn so với các khóa học tổng quát không?",
        "Các lớp học chuyên ngành của tôi có liên quan mật thiết với giáo viên không?"
      ],
      "responses": [
        "Kích thước lớp trong chuyên ngành mong muốn của bạn tại trường của chúng tôi thay đổi tùy thuộc vào các khóa học cụ thể và chương trình. Nói chung, các lớp học liên quan đến chuyên ngành thường nhỏ hơn, tạo điều kiện cho các tương tác cá nhân với giáo viên và các sinh viên khác."
      ],
      "context_set": ""
    },
    {
      "tag": "impacted_majors",
      "patterns": [
        "Ngành học mà tôi quan tâm có phải là một ngành học bị ảnh hưởng/quá mức đăng ký?",
        "Có bất kỳ hạn chế nào về việc đăng ký vào ngành học tôi quan tâm không?",
        "Có một số ngành học có hạn chế về việc đăng ký không?",
        "Những ngành học nào phổ biến nhất trong số sinh viên?",
        "Việc vào các ngành học cụ thể là cạnh tranh như thế nào?"
      ],
      "responses": [
        "Một số ngành học tại trường của chúng tôi có thể bị ảnh hưởng hoặc có hạn chế về việc đăng ký do nhu cầu cao. Tuy nhiên, chúng tôi luôn cố gắng đáp ứng các sở thích học thuật của sinh viên và cung cấp hỗ trợ trong việc khám phá các lộ trình thay thế nếu cần."
      ],
      "context_set": ""
    },
    {
      "tag": "guaranteed_major",
      "patterns": [
        "Tôi có được đảm bảo vào ngành học mà tôi quan tâm không?",
        "Có quy trình đăng ký trực tiếp cho ngành học tôi quan tâm không?",
        "Trường đại học xử lý thế nào với việc đăng ký vào một ngành học?",
        "Có thể thay đổi ngành học sau khi đã đăng ký không?",
        "Quy trình chuyển sang một ngành học khác là gì?"
      ],
      "responses": [
        "Mặc dù có thể có quy trình đăng ký trực tiếp vào một số ngành học nhất định, việc khai báo một ngành học thường diễn ra sau năm học đầu tiên. Trường của chúng tôi khuyến khích việc khám phá và cung cấp các cố vấn học thuật để hỗ trợ sinh viên chọn ngành học phù hợp dựa trên sở thích và mục tiêu của họ."
      ],
      "context_set": ""
    },
    {
      "tag": "major_admission_standards",
      "patterns": [
        "Có tiêu chuẩn tuyển sinh cao hơn cho một số ngành học không?",
        "Các ngành học khác nhau có các yêu cầu tuyển sinh khác nhau không?",
        "Các yêu cầu để vào các ngành học cạnh tranh là gì?",
        "Một số ngành học có sẵn có hạn chế không?",
        "GPA và các yêu cầu tiên quyết cần thiết cho các ngành học cụ thể là gì?"
      ],
      "responses": [
        "Một số ngành học tại trường của chúng tôi có thể có các yêu cầu tuyển sinh cụ thể do tính cạnh tranh cao hoặc sự có hạn về sự sẵn có. Những yêu cầu này có thể bao gồm các khóa học tiên quyết, ngưỡng GPA, hoặc một đánh giá hồ sơ, phụ thuộc vào chương trình."
      ],
      "context_set": ""
    },
    {
      "tag": "declaring_major",
      "patterns": [
        "Khi nào bạn phải khai báo một ngành học?",
        "Khung thời gian để khai báo một ngành học là gì?",
        "Có một hạn chót để chọn một ngành học không?",
        "Sinh viên có bắt buộc phải khai báo một ngành học ngay lập tức không?",
        "Có tính linh hoạt trong việc khai báo một ngành học không?"
      ],
      "responses": [
        "Tại trường của chúng tôi, sinh viên thường có tính linh hoạt để khám phá các lĩnh vực học thuật khác nhau trong năm học đầu tiên trước khi khai báo một ngành học. Khung thời gian để khai báo một ngành học có thể thay đổi, nhưng cố vấn học thuật luôn sẵn lòng hướng dẫn sinh viên qua quy trình này."
      ],
      "context_set": ""
    },
    {
      "tag": "time_spent_on_homework",
      "patterns": [
        "Sinh viên thường dành bao nhiêu thời gian cho bài tập về nhà?",
        "Có một số giờ học trung bình mỗi tuần không?",
        "Khối lượng công việc dự kiến cho sinh viên là gì?",
        "Sinh viên có bắt buộc phải hoàn thành nhiều bài tập ngoài lớp không?",
        "Tôi nên mong đợi dành bao nhiêu thời gian cho việc học mỗi tuần?"
      ],
      "responses": [
        "Số lượng thời gian mà sinh viên dành cho bài tập về nhà có thể thay đổi dựa trên khối lượng công việc học tập và chương trình học thuật của họ. Trung bình, sinh viên tại trường của chúng tôi dành khoảng [X] giờ mỗi tuần cho việc học và hoàn thành bài tập."
      ],
      "context_set": ""
    },
    {
      "tag": "writing_and_reading_expectations",
      "patterns": [
        "Mức độ viết và đọc được mong đợi là bao nhiêu?",
        "Có các yêu cầu cụ thể về việc đọc và viết cho một số ngành học không?",
        "Có các khóa học tập trung vào viết không?",
        "Trường đại học nhấn mạnh kỹ năng viết và đọc như thế nào?",
        "Sinh viên có cơ hội tham gia vào viết sáng tạo hoặc viết nghiên cứu không?"
      ],
      "responses": [
        "Tại trường của chúng tôi, việc viết và đọc là những yếu tố quan trọng của trải nghiệm học thuật. Nhiều ngành học tích hợp các khóa học tập trung vào viết và các nhiệm vụ dựa trên nghiên cứu. Sinh viên có cơ hội phát triển kỹ năng viết và đọc của mình thông qua các nhiệm vụ và dự án khác nhau."
      ],
      "context_set": ""
    },
    {
      "tag": "double_major",
      "patterns": [
        "Có thể học song ngành không?",
        "Sinh viên có thể theo học hai ngành cùng một lúc không?",
        "Làm thế nào để học song ngành?",
        "Có bất kỳ hạn chế nào về việc học song ngành không?",
        "Có nhiều sinh viên chọn học song ngành không?"
      ],
      "responses": [
        "Có, tại trường của chúng tôi, sinh viên có thể học song ngành. Sinh viên quan tâm đến việc học hai ngành cùng một lúc có thể làm việc với cố vấn học thuật để tạo ra một kế hoạch thực hiện yêu cầu cho cả hai ngành. Nhiều sinh viên chọn học song ngành để khám phá các sở thích đa dạng và tăng cường trải nghiệm học thuật của họ."
      ],
      "context_set": ""
    },
    {
      "tag": "academic_system",
      "patterns": [
        "Trường có hoạt động theo hệ thống học kỳ, học phần hoặc học kỳ không?",
        "Lịch học thuật như thế nào?",
        "Một năm học có bao nhiêu học kỳ?",
        "Trường có các kỳ học hè không?",
        "Thời lượng tiêu chuẩn của một học kỳ là bao nhiêu?"
      ],
      "responses": [
        "Trường của chúng tôi hoạt động theo hệ thống [học kỳ/học phần/học kỳ], với [X] học kỳ mỗi năm học. Chúng tôi cũng cung cấp các kỳ học hè cho sinh viên quan tâm đến việc tăng cường chương trình học hoặc học tín chỉ bổ sung."
      ],
      "context_set": ""
    },
    {
      "tag": "honors_college",
      "patterns": [
        "Trường có chương trình danh dự không?",
        "Có chương trình danh dự dành cho sinh viên có thành tích xuất sắc không?",
        "Lợi ích của chương trình danh dự là gì?",
        "Làm thế nào để sinh viên đủ điều kiện tham gia chương trình danh dự?",
        "Có yêu cầu bổ sung nào để tham gia chương trình danh dự không?"
      ],
      "responses": [
        "Có, trường của chúng tôi có chương trình danh dự cung cấp những trải nghiệm học thuật phong phú và một cộng đồng hỗ trợ cho sinh viên có thành tích xuất sắc. Sinh viên có thể đủ điều kiện tham gia chương trình danh dự dựa trên thành tích học thuật, điểm thi chuẩn và các tiêu chí khác. Các sinh viên tham gia chương trình danh dự thường có cơ hội tiếp cận các khóa học chuyên sâu, cơ hội nghiên cứu độc quyền và ưu tiên đăng ký học."
      ],
      "context_set": ""
    },

    {
      "tag": "honors_college_requirements",
      "patterns": [
        "Những yêu cầu học thuật để đủ điều kiện tham gia chương trình danh dự là gì?",
        "Điểm trung bình cộng (GPA) hoặc điểm thi chuẩn cần thiết để tham gia chương trình danh dự là bao nhiêu?",
        "Có các tiêu chí đủ điều kiện cụ thể cho chương trình danh dự không?",
        "Quy trình xin vào chương trình danh dự có cạnh tranh không?",
        "Sinh viên có phải duy trì các tiêu chuẩn học thuật nhất định trong chương trình danh dự không?"
      ],
      "responses": [
        "Để đủ điều kiện tham gia chương trình danh dự, sinh viên thường cần phải chứng minh thành tích học thuật xuất sắc, GPA cao và điểm thi chuẩn mạnh mẽ. Quy trình xin vào chương trình có tính cạnh tranh, nhưng ứng viên thành công sẽ nhận được những lợi ích từ việc trở thành một phần của một cộng đồng trí tuệ. Sinh viên danh dự có thể cần duy trì một GPA nhất định để tiếp tục tham gia vào chương trình."
      ],
      "context_set": ""
    },
    {
      "tag": "learning_community",
      "patterns": [
        "Trường có cộng đồng học tập hoặc các trải nghiệm học tập dành cho sinh viên năm nhất không?",
        "Có các chương trình được thiết kế để giúp sinh viên năm nhất chuyển tiếp sang đại học không?",
        "Có các nguồn lực nào để hỗ trợ sinh viên năm đầu?",
        "Có các chương trình hỗ trợ hoặc hỗ trợ từ bạn đồng học cho sinh viên năm nhất không?",
        "Trường làm thế nào để đảm bảo một sự chuyển tiếp mượt mà cho sinh viên mới?"
      ],
      "responses": [
        "Có, trường của chúng tôi cung cấp các cộng đồng học tập và các chương trình khác được thiết kế đặc biệt để hỗ trợ sinh viên năm nhất trong quá trình chuyển tiếp sang cuộc sống đại học. Các chương trình này cung cấp hỗ trợ học thuật, xã hội và cá nhân để giúp sinh viên mới cảm thấy chào đón và điều chỉnh mượt mà vào đại học."
      ],
      "context_set": ""
    },
    {
      "tag": "senior_capstone_experience",
      "patterns": [
        "Sinh viên cuối cùng có hoàn thành trải nghiệm hoặc dự án tổng kết không?",
        "Dự án tổng kết của sinh viên cuối cùng là gì?",
        "Trải nghiệm tổng kết này làm thế nào để góp phần vào việc học của sinh viên?",
        "Dự án tổng kết cuối cùng có bắt buộc đối với tất cả các ngành không?",
        "Loại hỗ trợ nào sinh viên nhận được trong quá trình dự án tổng kết của họ?"
      ],
      "responses": [
        "Có, sinh viên cuối cùng tại trường của chúng tôi thường hoàn thành trải nghiệm hoặc dự án tổng kết trong chuyên ngành của mình. Dự án tổng kết được thiết kế để giới thiệu sự thành thạo của sinh viên trong lĩnh vực học thuật của họ và thường liên quan đến nghiên cứu, công việc sáng tạo hoặc các dự án dựa trên cộng đồng. Các cố vấn khoa học cung cấp hướng dẫn và hỗ trợ trong suốt quá trình tổng kết."
      ],
      "context_set": ""
    },
    {
      "tag": "academic_advisory_system",
      "patterns": [
        "Hệ thống tư vấn học thuật của bạn như thế nào?",
        "Làm thế nào để tư vấn học thuật hoạt động?",
        "Sinh viên có được chỉ định cố vấn học thuật không?",
        "Có các cố vấn cụ thể cho các ngành học khác nhau không?",
        "Sinh viên có thể gặp gỡ cố vấn học thuật thường xuyên không?"
      ],
      "responses": [
        "Trường của chúng tôi có một hệ thống tư vấn học thuật mạnh mẽ. Sinh viên thường được chỉ định các cố vấn học thuật dựa trên chuyên ngành hoặc lĩnh vực quan tâm của họ. Các cố vấn học thuật hỗ trợ sinh viên trong việc lựa chọn môn học, lập kế hoạch học tập và điều hướng các nguồn lực của trường. Sinh viên có thể gặp gỡ cố vấn của mình thường xuyên để thảo luận về tiến trình học tập và mục tiêu của mình."
      ],
      "context_set": ""
    },
    {
      "tag": "professor_as_advisor",
      "patterns": [
        "Có thể chọn một giáo sư làm cố vấn học tập không?",
        "Sinh viên có thể chọn một giảng viên làm cố vấn học tập không?",
        "Giáo sư có sẵn sàng tư vấn một-một không?",
        "Làm thế nào để sinh viên yêu cầu một giáo sư cụ thể làm cố vấn của mình?",
        "Những lợi ích nào khi có giáo sư làm cố vấn học tập?"
      ],
      "responses": [
        "Có, sinh viên tại trường của chúng tôi có thể yêu cầu một giáo sư cụ thể làm cố vấn học tập, đặc biệt nếu họ có một sở thích nghiên cứu hoặc mục tiêu học tập cụ thể. Có một giáo sư làm cố vấn có thể cung cấp cho sinh viên hướng dẫn chuyên môn, sự cố vấn và những thông tin quý báu về con đường học tập và nghề nghiệp của họ."
      ],
      "context_set": ""
    },
    {
      "tag": "study_abroad_percentage",
      "patterns": [
        "Tỷ lệ sinh viên đi du học là bao nhiêu?",
        "Có nhiều sinh viên tham gia các chương trình du học không?",
        "Có cơ hội du học quốc tế không?",
        "Có những loại chương trình du học nào?",
        "Du học nâng cao trải nghiệm đại học của sinh viên như thế nào?"
      ],
      "responses": [
        "Khoảng [X]% sinh viên tại trường chúng tôi tham gia các chương trình du học. Chúng tôi cung cấp một loạt các cơ hội du học đa dạng giúp sinh viên hòa mình vào các nền văn hóa khác nhau, có được cái nhìn toàn cầu và nâng cao kỹ năng ngôn ngữ. Du học là một trải nghiệm phong phú giúp phát triển cá nhân và nhận thức toàn cầu."
      ],
      "context_set": ""
    },
    {
      "tag": "study_abroad_intended_major",
      "patterns": [
        "Sinh viên trong chuyên ngành của tôi có thường đi du học không?",
        "Có những chương trình du học cụ thể liên quan đến chuyên ngành của tôi không?",
        "Du học có lợi ích gì cho chuyên ngành của tôi?",
        "Sinh viên trong các chuyên ngành nào dễ dàng đi du học hơn?",
        "Có hỗ trợ nào cho các cơ hội du học theo chuyên ngành không?"
      ],
      "responses": [
        "Có, sinh viên trong nhiều chuyên ngành khác nhau, bao gồm [chuyên ngành của bạn], có cơ hội đi du học. Trường chúng tôi cung cấp các chương trình du học theo chuyên ngành phù hợp với sở thích học tập và nâng cao hiểu biết của sinh viên về lĩnh vực của họ. Du học có thể mang lại những góc nhìn và trải nghiệm độc đáo bổ sung cho hành trình học tập."
      ],
      "context_set": ""
    },
    {
      "tag": "undergraduate_research_opportunities",
      "patterns": [
        "Có những cơ hội nào cho nghiên cứu đại học?",
        "Có cơ hội nghiên cứu nào dành cho sinh viên đại học không?",
        "Sinh viên đại học có thể tham gia các dự án nghiên cứu không?",
        "Làm thế nào để tôi tham gia vào nghiên cứu khi còn là sinh viên đại học?",
        "Những lợi ích của nghiên cứu đại học là gì?"
      ],
      "responses": [
        "Nghiên cứu đại học được khuyến khích mạnh mẽ tại trường chúng tôi, và có nhiều cơ hội cho sinh viên tham gia vào các dự án nghiên cứu. Sinh viên có thể làm việc chặt chẽ với các giảng viên cố vấn, tham gia các hội nghị nghiên cứu và đóng góp vào những phát hiện tiên tiến trong nhiều lĩnh vực khác nhau. Nghiên cứu đại học mang lại những trải nghiệm quý báu giúp nâng cao tư duy phản biện và chuẩn bị cho sinh viên cho sự nghiệp tương lai và học tiếp lên cao."
      ],
      "context_set": ""
    },
    {
      "tag": "undergraduate_research_participation",
      "patterns": [
        "Bao nhiêu sinh viên tham gia vào nghiên cứu đại học?",
        "Nghiên cứu đại học có phổ biến trong khuôn viên không?",
        "Có nhiều sinh viên tham gia vào các dự án nghiên cứu không?",
        "Tỷ lệ phần trăm sinh viên đại học tham gia vào nghiên cứu là bao nhiêu?",
        "Những khoa nào có tỷ lệ tham gia nghiên cứu cao nhất?"
      ],
      "responses": [
        "Một tỷ lệ đáng kể sinh viên đại học tại trường chúng tôi tích cực tham gia vào nghiên cứu đại học. Chúng tôi có nhiều khoa khác nhau nơi sinh viên có thể hợp tác với các giảng viên trong các dự án nghiên cứu, làm cho nghiên cứu trở thành một phần phổ biến và phong phú của trải nghiệm học tập."
      ],
      "context_set": ""
    },
    {
      "tag": "departments_with_research",
      "patterns": [
        "Những khoa nào cung cấp cơ hội nghiên cứu đại học?",
        "Có những khoa nào ưu tiên nghiên cứu?",
        "Những lĩnh vực học thuật nào cung cấp cơ hội nghiên cứu?",
        "Sinh viên trong tất cả các chuyên ngành có thể tham gia vào nghiên cứu không?",
        "Có hỗ trợ nào cho nghiên cứu trong các khoa khác nhau không?"
      ],
      "responses": [
        "Cơ hội nghiên cứu đại học có sẵn trong nhiều khoa tại trường chúng tôi. Sinh viên trong hầu hết các chuyên ngành đều có cơ hội tham gia vào các dự án nghiên cứu. Mỗi khoa cung cấp các nguồn lực, cố vấn và cơ sở vật chất để hỗ trợ các nỗ lực nghiên cứu của sinh viên."
      ],
      "context_set": ""
    },
    {
      "tag": "academic_credit_transfer",
      "patterns": [
        "Nếu muốn chuyển tiếp: làm thế nào để biết bao nhiêu tín chỉ học thuật sẽ được chuyển tiếp?",
        "Quá trình chuyển tiếp tín chỉ cho sinh viên chuyển trường như thế nào?",
        "Có hướng dẫn cụ thể nào cho việc chuyển tiếp tín chỉ không?",
        "Tín chỉ được đánh giá như thế nào trong quá trình chuyển tiếp?",
        "Có tối đa số lượng tín chỉ có thể được chuyển tiếp không?"
      ],
      "responses": [
        "Đối với sinh viên chuyển trường, trường chúng tôi có một quy trình đánh giá tín chỉ để xác định số lượng tín chỉ học thuật sẽ được chuyển tiếp từ trường trước của họ. Trường đánh giá sự tương đương của các khóa học đã học ở trường trước và cấp tín chỉ chuyển tiếp tương ứng. Có thể có một giới hạn tối đa về số lượng tín chỉ có thể được chuyển tiếp."
      ],
      "context_set": ""
    },
    {
      "tag": "four_year_graduation_rate",
      "patterns": [
        "Tỷ lệ tốt nghiệp trong bốn năm của trường là bao nhiêu?",
        "Bao nhiêu sinh viên tốt nghiệp trong vòng bốn năm?",
        "Tỷ lệ tốt nghiệp trong bốn năm có cao không?",
        "Tỷ lệ phần trăm sinh viên hoàn thành bằng cấp trong bốn năm là bao nhiêu?",
        "Trường hỗ trợ sinh viên tốt nghiệp đúng hạn như thế nào?"
      ],
      "responses": [
        "Trường chúng tôi tự hào có tỷ lệ tốt nghiệp trong bốn năm cao khoảng [X]%. Chúng tôi cung cấp nhiều nguồn lực khác nhau, chẳng hạn như cố vấn học tập và kế hoạch bằng cấp cá nhân hóa, để hỗ trợ sinh viên duy trì lộ trình và tốt nghiệp đúng hạn."
      ],
      "context_set": ""
    },
    {
      "tag": "graduation_in_four_years",
      "patterns": [
        "Yêu cầu để tốt nghiệp trong bốn năm là gì?",
        "Làm thế nào để sinh viên lập kế hoạch hoàn thành bằng cấp trong bốn năm?",
        "Có những chiến lược nào để đảm bảo tốt nghiệp đúng hạn?",
        "Sinh viên có thường tốt nghiệp trong bốn năm không?",
        "Sinh viên có thể hoàn thành hai chuyên ngành trong bốn năm không?"
      ],
      "responses": [
        "Tốt nghiệp trong bốn năm thường đòi hỏi lập kế hoạch học tập cẩn thận, hoàn thành các yêu cầu tín chỉ, và duy trì tiến độ học tập. Sinh viên có thể làm việc chặt chẽ với cố vấn học tập để lập kế hoạch lịch học và đảm bảo họ đáp ứng tất cả các yêu cầu tốt nghiệp. Mặc dù tốt nghiệp trong bốn năm là điều phổ biến với nhiều sinh viên, một số có thể mất thời gian lâu hơn, đặc biệt nếu theo đuổi hai chuyên ngành hoặc tham gia các chương trình hợp tác. Tuy nhiên, với kế hoạch phù hợp, nhiều sinh viên hoàn thành bằng cấp của họ trong vòng bốn năm."
      ],
      "context_set": ""
    },
    {
      "tag": "freshmen_retention_rate",
      "patterns": [
        "Tỷ lệ sinh viên năm nhất quay lại vào năm hai là bao nhiêu?",
        "Tỷ lệ giữ chân sinh viên năm nhất có cao không?",
        "Những yếu tố nào đóng góp vào tỷ lệ giữ chân cao?",
        "Trường hỗ trợ sinh viên năm nhất trong việc chuyển tiếp vào đời sống đại học như thế nào?",
        "Có các chương trình định hướng cho sinh viên năm nhất không?"
      ],
      "responses": [
        "Trường chúng tôi thường có tỷ lệ giữ chân sinh viên năm nhất cao, cho thấy một tỷ lệ đáng kể sinh viên năm nhất chọn quay lại vào năm hai. Trường cung cấp hỗ trợ toàn diện cho sinh viên năm nhất trong việc chuyển tiếp vào đời sống đại học, bao gồm các chương trình định hướng, cố vấn học tập, dịch vụ hỗ trợ sinh viên, và một cộng đồng trường học thân thiện. Những trải nghiệm hấp dẫn và hỗ trợ trong năm đầu tiên có thể góp phần vào trải nghiệm đại học tích cực và khuyến khích sinh viên tiếp tục học tập tại trường."
      ],
      "context_set": ""
    },
    {
      "tag": "reasons_students_leave",
      "patterns": [
        "Những lý do chính khiến sinh viên rời trường sớm là gì?",
        "Những yếu tố nào đóng góp vào việc sinh viên bỏ học?",
        "Có những thách thức chung nào sinh viên gặp phải dẫn đến việc rời trường?",
        "Trường giải quyết các vấn đề có thể dẫn đến việc sinh viên rời trường như thế nào?",
        "Có hỗ trợ nào dành cho sinh viên đang xem xét việc rời trường không?"
      ],
      "responses": [
        "Việc bỏ học có thể do nhiều yếu tố khác nhau, và các trường học làm việc để giải quyết những vấn đề này một cách chủ động. Một số lý do phổ biến khiến sinh viên rời trường sớm bao gồm các vấn đề tài chính, thách thức học tập, hoàn cảnh cá nhân, và nỗi nhớ nhà. Trường cung cấp các dịch vụ hỗ trợ, tư vấn, và tài nguyên để hỗ trợ sinh viên gặp khó khăn và tìm kiếm các giải pháp tiềm năng để ngăn chặn việc rời trường sớm."
      ],
      "context_set": ""
    },
    {
      "tag": "academic_advising_system",
      "patterns": [
        "Làm thế nào để trường cung cấp lời khuyên học tập cho sinh viên?",
        "Có hệ thống cố vấn học tập không?",
        "Sinh viên được kết nối với cố vấn học tập như thế nào?",
        "Những tài nguyên nào có sẵn thông qua cố vấn học tập?",
        "Sinh viên có thể tìm lời khuyên về việc chọn khóa học và khám phá chuyên ngành không?"
      ],
      "responses": [
        "Trường chúng tôi có hệ thống cố vấn học tập mạnh mẽ để hỗ trợ sinh viên suốt hành trình học tập của họ. Sinh viên thường được kết nối với cố vấn học tập dựa trên chuyên ngành đã khai báo của họ hoặc thông qua cố vấn chung cho sinh viên chưa quyết định chuyên ngành. Các cố vấn học tập cung cấp hướng dẫn về việc chọn khóa học, khám phá chuyên ngành, lập kế hoạch học tập, và tài nguyên trong trường. Sinh viên được khuyến khích gặp cố vấn của họ thường xuyên để thảo luận về tiến độ học tập và nhận lời khuyên cá nhân hóa."
      ],
      "context_set": ""
    },
    {
      "tag": "professor_as_advisor",
      "patterns": [
        "Có thể chọn một giáo sư làm cố vấn học tập không?",
        "Sinh viên có thể chọn một giáo sư làm cố vấn học tập không?",
        "Có lợi ích gì khi có một giáo sư làm cố vấn?",
        "Sinh viên có thể tiếp cận một giáo sư để làm cố vấn như thế nào?",
        "Vai trò của cố vấn là giáo sư trong hướng dẫn học tập và nghề nghiệp là gì?"
      ],
      "responses": [
        "Có, trong một số trường hợp, sinh viên có thể có cơ hội chọn một giáo sư làm cố vấn học tập. Có một giáo sư làm cố vấn có thể cung cấp cho sinh viên hướng dẫn chuyên môn trong lĩnh vực học tập của họ, cơ hội nghiên cứu, và những thông tin về con đường nghề nghiệp tiềm năng. Để tiếp cận một giáo sư để xin tư vấn, sinh viên có thể bày tỏ sự quan tâm của mình trong giờ làm việc hoặc qua email. Các cố vấn là giáo sư có thể giúp sinh viên với kế hoạch học tập, cơ hội nghiên cứu, và lời khuyên liên quan đến nghề nghiệp."
      ],
      "context_set": ""
    },
    {
      "tag": "percentage_students_studies_abroad",
      "patterns": [
        "Tỷ lệ phần trăm sinh viên đi du học là bao nhiêu?",
        "Có nhiều sinh viên tham gia các chương trình du học không?",
        "Cơ hội du học quốc tế phổ biến như thế nào?",
        "Có các lựa chọn hỗ trợ tài chính cho việc du học không?",
        "Có những loại chương trình du học nào?"
      ],
      "responses": [
        "Một tỷ lệ đáng kể sinh viên của trường chúng tôi chọn đi du học vào một thời điểm nào đó trong hành trình học tập của họ. Các cơ hội du học quốc tế rất phổ biến do những trải nghiệm phong phú mà chúng mang lại. Trường thường cung cấp các lựa chọn hỗ trợ tài chính, học bổng, và trợ cấp để hỗ trợ sinh viên trong việc tài trợ cho các trải nghiệm du học. Sinh viên có thể khám phá các chương trình du học khác nhau, bao gồm các trao đổi học kỳ, các chuyến tham quan ngắn hạn, và thực tập quốc tế."
      ],
      "context_set": ""
    },
    {
      "tag": "participating_students_undergrad_research",
      "patterns": [
        "Bao nhiêu sinh viên tham gia vào nghiên cứu đại học?",
        "Nghiên cứu đại học có phổ biến tại trường không?",
        "Tỷ lệ phần trăm sinh viên tham gia vào các dự án nghiên cứu là bao nhiêu?",
        "Có các hội thảo nghiên cứu hoặc sự kiện để trưng bày nghiên cứu của sinh viên không?",
        "Những lợi ích nào sinh viên có được từ việc tham gia vào nghiên cứu đại học?"
      ],
      "responses": [
        "Một tỷ lệ lớn sinh viên tại trường chúng tôi tích cực tham gia vào nghiên cứu đại học. Trường thường tổ chức các hội thảo nghiên cứu hoặc các sự kiện để trưng bày các dự án nghiên cứu của sinh viên và tôn vinh những đóng góp của họ cho cộng đồng học thuật. Tham gia vào nghiên cứu đại học cung cấp cho sinh viên những trải nghiệm quý báu, sự cố vấn từ các giảng viên, phát triển kỹ năng, và cơ hội đóng góp vào sự tiến bộ của kiến thức trong lĩnh vực của họ."
      ],
      "context_set": ""
    },
    {
      "tag": "departments_offer_undergrad_research",
      "patterns": [
        "Những khoa nào cung cấp nghiên cứu đại học?",
        "Những lĩnh vực học thuật nào nhấn mạnh cơ hội nghiên cứu?",
        "Cơ hội nghiên cứu có sẵn trong tất cả các ngành học không?",
        "Sinh viên từ bất kỳ chuyên ngành nào có thể tham gia vào nghiên cứu không?",
        "Làm thế nào để sinh viên tìm cơ hội nghiên cứu trong lĩnh vực của họ?"
      ],
      "responses": [
        "Cơ hội nghiên cứu đại học có sẵn trong nhiều ngành học tại trường chúng tôi. Nhiều khoa và giảng viên khuyến khích sinh viên từ tất cả các chuyên ngành tham gia vào các dự án nghiên cứu. Để tìm cơ hội nghiên cứu trong lĩnh vực của mình, sinh viên có thể liên hệ với các giảng viên trong khoa của họ, khám phá các trung tâm nghiên cứu, tham dự các sự kiện hướng về nghiên cứu, và tham khảo văn phòng nghiên cứu đại học của trường để được hướng dẫn."
      ],
      "context_set": ""
    },
    {
      "tag": "academic_credits_transfer",
      "patterns": [
        "Nếu bạn muốn chuyển trường: làm thế nào để biết bao nhiêu tín chỉ học tập sẽ được chuyển?",
        "Có hướng dẫn cụ thể nào về việc chuyển tín chỉ không?",
        "Quy trình chuyển tín chỉ từ trường khác là gì?",
        "Trường có hệ thống đánh giá tín chỉ chuyển tiếp không?",
        "Các khóa học trước đây của tôi có được tính vào yêu cầu bằng cấp không?"
      ],
      "responses": [
        "Nếu bạn đang xem xét việc chuyển đến trường chúng tôi, chúng tôi có hệ thống đánh giá tín chỉ chuyển tiếp để đánh giá các khóa học trước đây của bạn. Trường sẽ xem xét bảng điểm của bạn và xác định tín chỉ nào sẽ được chuyển và tính vào yêu cầu bằng cấp của bạn. Hướng dẫn và quy trình cụ thể về việc chuyển tín chỉ có thể khác nhau, nhưng văn phòng tuyển sinh chuyển tiếp hoặc cố vấn học tập của trường có thể cung cấp hướng dẫn trong suốt quá trình đánh giá."
      ],
      "context_set": ""
    },
    {
      "tag": "four_year_graduation_rate",
      "patterns": [
        "Tỷ lệ tốt nghiệp trong bốn năm của trường bạn là bao nhiêu?",
        "Có bao nhiêu sinh viên tốt nghiệp trong bốn năm?",
        "Tỷ lệ tốt nghiệp trong bốn năm ở trường bạn có cao không?",
        "Những yếu tố nào đóng góp vào tỷ lệ tốt nghiệp trong bốn năm cao?",
        "Có những tài nguyên nào hỗ trợ tốt nghiệp đúng hạn không?"
      ],
      "responses": [
        "Trường chúng tôi có tỷ lệ tốt nghiệp trong bốn năm cao, điều này cho thấy cam kết của trường trong việc hỗ trợ thành công học tập và hoàn thành bằng cấp đúng hạn của sinh viên. Các yếu tố đóng góp vào tỷ lệ tốt nghiệp trong bốn năm cao bao gồm cố vấn học tập mạnh mẽ, các khóa học dễ tiếp cận, và môi trường trường học hỗ trợ. Trường có thể cung cấp các tài nguyên để giúp sinh viên duy trì tiến độ học tập và tốt nghiệp đúng hạn."
      ],
      "context_set": ""
    },
    {
      "tag": "graduation_in_four_years",
      "patterns": [
        "Yêu cầu để tốt nghiệp trong bốn năm là gì?",
        "Làm thế nào để sinh viên lập kế hoạch hoàn thành bằng cấp trong bốn năm?",
        "Có những chiến lược nào để đảm bảo tốt nghiệp đúng hạn?",
        "Sinh viên có thường tốt nghiệp trong bốn năm không?",
        "Sinh viên có thể hoàn thành hai chuyên ngành trong bốn năm không?"
      ],
      "responses": [
        "Tốt nghiệp trong bốn năm thường đòi hỏi lập kế hoạch học tập cẩn thận, hoàn thành các yêu cầu tín chỉ, và duy trì tiến độ học tập. Sinh viên có thể làm việc chặt chẽ với cố vấn học tập để lập kế hoạch lịch học và đảm bảo họ đáp ứng tất cả các yêu cầu tốt nghiệp. Mặc dù tốt nghiệp trong bốn năm là điều phổ biến với nhiều sinh viên, một số có thể mất thời gian lâu hơn, đặc biệt nếu theo đuổi hai chuyên ngành hoặc tham gia các chương trình hợp tác. Tuy nhiên, với kế hoạch phù hợp, nhiều sinh viên hoàn thành bằng cấp của họ trong vòng bốn năm."
      ],
      "context_set": ""
    },
    {
      "tag": "freshmen_retention_rate",
      "patterns": [
        "Tỷ lệ sinh viên năm nhất quay lại vào năm hai là bao nhiêu?",
        "Tỷ lệ giữ chân sinh viên năm nhất có cao không?",
        "Những yếu tố nào đóng góp vào tỷ lệ giữ chân cao?",
        "Trường hỗ trợ sinh viên năm nhất trong việc chuyển tiếp vào đời sống đại học như thế nào?",
        "Có các chương trình định hướng cho sinh viên năm nhất không?"
      ],
      "responses": [
        "Trường chúng tôi thường có tỷ lệ giữ chân sinh viên năm nhất cao, cho thấy một tỷ lệ đáng kể sinh viên năm nhất chọn quay lại vào năm hai. Trường cung cấp hỗ trợ toàn diện cho sinh viên năm nhất trong việc chuyển tiếp vào đời sống đại học, bao gồm các chương trình định hướng, cố vấn học tập, dịch vụ hỗ trợ sinh viên, và một cộng đồng trường học thân thiện. Những trải nghiệm hấp dẫn và hỗ trợ trong năm đầu tiên có thể góp phần vào trải nghiệm đại học tích cực và khuyến khích sinh viên tiếp tục học tập tại trường."
      ],
      "context_set": ""
    },
    {
      "tag": "reasons_students_leave",
      "patterns": [
        "Những lý do chính khiến sinh viên rời trường sớm là gì?",
        "Những yếu tố nào đóng góp vào việc sinh viên bỏ học?",
        "Có những thách thức chung nào sinh viên gặp phải dẫn đến việc rời trường?",
        "Trường giải quyết các vấn đề có thể dẫn đến việc sinh viên rời trường như thế nào?",
        "Có hỗ trợ nào dành cho sinh viên đang xem xét việc rời trường không?"
      ],
      "responses": [
        "Việc bỏ học có thể do nhiều yếu tố khác nhau, và các trường học làm việc để giải quyết những vấn đề này một cách chủ động. Một số lý do phổ biến khiến sinh viên rời trường sớm bao gồm các vấn đề tài chính, thách thức học tập, hoàn cảnh cá nhân, và nỗi nhớ nhà. Trường cung cấp các dịch vụ hỗ trợ, tư vấn, và tài nguyên để hỗ trợ sinh viên gặp khó khăn và tìm kiếm các giải pháp tiềm năng để ngăn chặn việc rời trường sớm."
      ],
      "context_set": ""
    },
    {
      "tag": "academic_advising_system",
      "patterns": [
        "Làm thế nào để trường cung cấp lời khuyên học tập cho sinh viên?",
        "Có hệ thống cố vấn học tập không?",
        "Sinh viên được kết nối với cố vấn học tập như thế nào?",
        "Những tài nguyên nào có sẵn thông qua cố vấn học tập?",
        "Sinh viên có thể tìm lời khuyên về việc chọn khóa học và khám phá chuyên ngành không?"
      ],
      "responses": [
        "Trường chúng tôi có hệ thống cố vấn học tập mạnh mẽ để hỗ trợ sinh viên suốt hành trình học tập của họ. Sinh viên thường được kết nối với cố vấn học tập dựa trên chuyên ngành đã khai báo của họ hoặc thông qua cố vấn chung cho sinh viên chưa quyết định chuyên ngành. Các cố vấn học tập cung cấp hướng dẫn về việc chọn khóa học, khám phá chuyên ngành, lập kế hoạch học tập, và tài nguyên trong trường. Sinh viên được khuyến khích gặp cố vấn của họ thường xuyên để thảo luận về tiến độ học tập và nhận lời khuyên cá nhân hóa."
      ],
      "context_set": ""
    },
    {
      "tag": "professor_as_advisor",
      "patterns": [
        "Có thể chọn một giáo sư làm cố vấn học tập không?",
        "Sinh viên có thể chọn một giáo sư làm cố vấn học tập không?",
        "Có lợi ích gì khi có một giáo sư làm cố vấn?",
        "Sinh viên có thể tiếp cận một giáo sư để làm cố vấn như thế nào?",
        "Vai trò của cố vấn là giáo sư trong hướng dẫn học tập và nghề nghiệp là gì?"
      ],
      "responses": [
        "Có, trong một số trường hợp, sinh viên có thể có cơ hội chọn một giáo sư làm cố vấn học tập. Có một giáo sư làm cố vấn có thể cung cấp cho sinh viên hướng dẫn chuyên môn trong lĩnh vực học tập của họ, cơ hội nghiên cứu, và những thông tin về con đường nghề nghiệp tiềm năng. Để tiếp cận một giáo sư để xin tư vấn, sinh viên có thể bày tỏ sự quan tâm của mình trong giờ làm việc hoặc qua email. Các cố vấn là giáo sư có thể giúp sinh viên với kế hoạch học tập, cơ hội nghiên cứu, và lời khuyên liên quan đến nghề nghiệp."
      ],
      "context_set": ""
    },
    {
      "tag": "percentage_students_studies_abroad",
      "patterns": [
        "Bao nhiêu phần trăm sinh viên học ở nước ngoài?",
        "Có nhiều sinh viên tham gia các chương trình du học không?",
        "Cơ hội học tập quốc tế có phổ biến không?",
        "Có các lựa chọn hỗ trợ tài chính cho việc du học không?",
        "Các loại chương trình du học nào có sẵn?"
      ],
      "responses": [
        "Một tỷ lệ đáng kể sinh viên của trường chọn học ở nước ngoài vào một thời điểm nào đó trong hành trình học tập của họ. Cơ hội học tập quốc tế rất phổ biến vì những trải nghiệm phong phú mà chúng mang lại. Trường thường cung cấp các lựa chọn hỗ trợ tài chính, học bổng, và trợ cấp để hỗ trợ sinh viên trong việc tài trợ cho trải nghiệm du học của họ. Sinh viên có thể khám phá các chương trình du học khác nhau, bao gồm các chương trình trao đổi dài hạn, các chuyến du học ngắn hạn, và thực tập quốc tế."
      ],
      "context_set": ""
    },
    {
      "tag": "undergraduate_research_opportunities",
      "patterns": [
        "Có những cơ hội nào cho nghiên cứu bậc đại học?",
        "Trường có chú trọng đến nghiên cứu bậc đại học không?",
        "Có các chương trình nghiên cứu cho sinh viên đại học không?",
        "Làm thế nào để sinh viên tham gia vào các dự án nghiên cứu?",
        "Có cơ hội nghiên cứu cho tất cả các ngành học không?"
      ],
      "responses": [
        "Trường chúng tôi đặt sự nhấn mạnh mạnh mẽ vào nghiên cứu bậc đại học, và có nhiều cơ hội cho sinh viên tham gia vào các dự án nghiên cứu. Các chương trình nghiên cứu, dự án do giảng viên dẫn dắt, và các trung tâm nghiên cứu có sẵn để hỗ trợ sinh viên đại học trong các nỗ lực nghiên cứu của họ. Sinh viên có thể khám phá các cơ hội nghiên cứu trong các ngành học khác nhau, và các giảng viên thường khuyến khích sinh viên của họ tham gia vào các trải nghiệm nghiên cứu thực hành."
      ],
      "context_set": ""
    },
    {
      "tag": "participating_students_undergrad_research",
      "patterns": [
        "Bao nhiêu sinh viên tham gia vào nghiên cứu bậc đại học?",
        "Nghiên cứu bậc đại học có phổ biến tại trường không?",
        "Bao nhiêu phần trăm sinh viên tham gia vào các dự án nghiên cứu?",
        "Có các hội thảo hoặc sự kiện nào để trưng bày nghiên cứu của sinh viên không?",
        "Sinh viên nhận được lợi ích gì khi tham gia vào nghiên cứu bậc đại học?"
      ],
      "responses": [
        "Một tỷ lệ đáng kể sinh viên tại trường chúng tôi tích cực tham gia vào nghiên cứu bậc đại học. Trường thường tổ chức các hội thảo nghiên cứu hoặc các sự kiện để trưng bày các dự án nghiên cứu của sinh viên và tôn vinh những đóng góp của họ cho cộng đồng học thuật. Tham gia vào nghiên cứu bậc đại học mang lại cho sinh viên những trải nghiệm quý giá, sự hướng dẫn từ các giảng viên, phát triển kỹ năng, và cơ hội đóng góp vào sự phát triển của kiến thức trong lĩnh vực của họ."
      ],
      "context_set": ""
    },
    {
      "tag": "semester_abroad",
      "patterns": [
        "Sinh viên có thể học một học kỳ ở nước ngoài không?",
        "Du học có phải là một lựa chọn cho sinh viên không?",
        "Trường có cung cấp các chương trình du học không?",
        "Hãy kể cho tôi về các cơ hội trải nghiệm quốc tế.",
        "Sinh viên có thể học ở những quốc gia nào trong một học kỳ?"
      ],
      "responses": [
        "Có, trường cung cấp các chương trình học kỳ ở nước ngoài cho sinh viên muốn học tập ở các quốc gia khác nhau và trải nghiệm một nền văn hóa khác. Sinh viên có thể chọn từ nhiều trường đối tác và chương trình trao đổi để làm phong phú thêm trải nghiệm giáo dục của mình."
      ],
      "context_set": ""
    },
    {
      "tag": "freshmen_mentoring_program",
      "patterns": [
        "Có chương trình cố vấn nào cho sinh viên năm nhất để giúp họ chuyển tiếp vào đời sống đại học không?",
        "Trường hỗ trợ sinh viên mới nhập học như thế nào?",
        "Có những người cố vấn đồng môn cho sinh viên năm nhất không?",
        "Hãy kể cho tôi về các chương trình giúp sinh viên năm nhất thích nghi với đại học."
      ],
      "responses": [
        "Có, chúng tôi có một chương trình cố vấn toàn diện được thiết kế đặc biệt để hỗ trợ và hướng dẫn sinh viên năm nhất trong quá trình chuyển tiếp vào đời sống đại học. Các sáng kiến cố vấn đồng môn của chúng tôi cung cấp các tài nguyên quý giá, lời khuyên, và cảm giác cộng đồng cho sinh viên mới khi họ điều hướng năm đầu tiên tại trường."
      ],
      "context_set": ""
    },
    {
      "tag": "campus_safety_security",
      "patterns": [
        "Hãy kể cho tôi về các biện pháp an ninh và dịch vụ bảo vệ tại trường.",
        "Khuôn viên trường có an toàn cho sinh viên không?",
        "Có những tài nguyên bảo vệ nào có sẵn tại trường?",
        "Trường đảm bảo an toàn cho sinh viên như thế nào?"
      ],
      "responses": [
        "An toàn và bảo vệ của sinh viên là điều quan trọng nhất. Khuôn viên trường được trang bị nhiều biện pháp an ninh, bao gồm các lối đi có ánh sáng tốt, tuần tra bảo vệ 24/7, các trạm gọi khẩn cấp, và hệ thống kiểm soát truy cập. Chúng tôi cũng có một bộ phận cảnh sát trường làm việc chăm chỉ để duy trì một môi trường an toàn cho tất cả các thành viên của cộng đồng chúng tôi."
      ],
      "context_set": ""
    },
    {
      "tag": "student_organizations",
      "patterns": [
        "Có những loại tổ chức sinh viên nào cho sinh viên tham gia?",
        "Có các câu lạc bộ và hoạt động ngoại khóa trong khuôn viên không?",
        "Hãy kể cho tôi về các cơ hội tham gia của sinh viên.",
        "Sinh viên có thể tham gia vào đời sống khuôn viên như thế nào?"
      ],
      "responses": [
        "Chúng tôi cung cấp một loạt các tổ chức sinh viên và câu lạc bộ cho nhiều sở thích và đam mê khác nhau. Sinh viên có thể chọn từ các nhóm học thuật, văn hóa, thể thao, xã hội, và phục vụ cộng đồng. Tham gia vào các tổ chức này là một cách tuyệt vời để kết bạn, phát triển kỹ năng lãnh đạo, và tham gia vào đời sống khuôn viên."
      ],
      "context_set": ""
    },
    {
      "tag": "experiential_learning_internships",
      "patterns": [
        "Hãy kể cho tôi về cách tiếp cận của trường đối với học tập thực nghiệm và thực tập.",
        "Có các cơ hội học tập thực hành không?",
        "Trường kết nối sinh viên với các cơ hội thực tập như thế nào?",
        "Sinh viên có được kinh nghiệm làm việc thực tế không?"
      ],
      "responses": [
        "Học tập thực nghiệm và thực tập là những phần không thể thiếu trong triết lý giáo dục của chúng tôi. Chúng tôi tin tưởng vào việc cung cấp cho sinh viên những trải nghiệm thực tế bổ sung cho việc học tập học thuật của họ. Thông qua các chương trình thực tập phong phú và hợp tác với các nhà lãnh đạo ngành công nghiệp, sinh viên có cơ hội để có được kinh nghiệm làm việc thực tế và xây dựng các kết nối quý giá cho sự nghiệp tương lai của họ."
      ],
      "context_set": ""
    },
    {
      "tag": "entrepreneurship_resources_clubs",
      "patterns": [
        "Có tài nguyên hoặc câu lạc bộ đặc biệt nào cho sinh viên quan tâm đến khởi nghiệp không?",
        "Hãy kể cho tôi về các cơ hội cho những người có tham vọng khởi nghiệp.",
        "Có trung tâm khởi nghiệp tại trường không?",
        "Trường hỗ trợ sinh viên với tham vọng khởi nghiệp như thế nào?"
      ],
      "responses": [
        "Có, chúng tôi có các tài nguyên và câu lạc bộ dành riêng cho sinh viên quan tâm đến khởi nghiệp. Trung tâm khởi nghiệp của chúng tôi cung cấp sự cố vấn, các hội thảo, cơ hội tài trợ, và các sự kiện kết nối cho những người có tham vọng khởi nghiệp. Sinh viên cũng có thể tham gia vào các câu lạc bộ liên quan đến khởi nghiệp để hợp tác trong các dự án sáng tạo và học hỏi từ các chuyên gia giàu kinh nghiệm trong lĩnh vực này."
      ],
      "context_set": ""
    },
    {
      "tag": "sustainability_initiatives",
      "patterns": [
        "Hãy cho tôi biết về các sáng kiến bền vững và nỗ lực xanh của trường.",
        "Trường có cam kết với sự bền vững môi trường không?",
        "Trường đã thực hiện những bước nào để trở nên thân thiện với môi trường hơn?",
        "Có các sáng kiến xanh nào trên khuôn viên trường không?"
      ],
      "responses": [
        "Sự bền vững môi trường là một trọng tâm chính của trường chúng tôi. Chúng tôi đã triển khai nhiều sáng kiến xanh khác nhau, bao gồm các chương trình tái chế, các tòa nhà tiết kiệm năng lượng, các lựa chọn giao thông bền vững, và các chiến dịch nâng cao nhận thức toàn trường. Trường của chúng tôi cam kết giảm thiểu tác động môi trường và thúc đẩy một khuôn viên xanh hơn vì lợi ích của các thế hệ tương lai."
      ],
      "context_set": ""
    },
    {
      "tag": "performing_arts_music_facilities",
      "patterns": [
        "Các cơ sở vật chất cho nghệ thuật biểu diễn và âm nhạc của trường như thế nào?",
        "Trường có trung tâm nghệ thuật biểu diễn không?",
        "Có không gian thực hành cho nhạc sĩ không?",
        "Hãy cho tôi biết về các địa điểm biểu diễn trên khuôn viên trường."
      ],
      "responses": [
        "Trường chúng tôi tự hào với các cơ sở vật chất hiện đại cho nghệ thuật biểu diễn và âm nhạc. Chúng tôi có một trung tâm nghệ thuật biểu diễn chuyên dụng tổ chức các buổi biểu diễn kịch, hòa nhạc và nhảy múa. Nhạc sĩ có thể sử dụng các phòng thực hành và phòng thu âm để trau dồi kỹ năng và hợp tác trong các dự án âm nhạc. Khuôn viên trường là một trung tâm sôi động cho sự biểu diễn nghệ thuật và các buổi biểu diễn văn hóa."
      ],
      "context_set": ""
    },
    {
      "tag": "class_sizes",
      "patterns": [
        "Kích thước lớp học như thế nào?",
        "Các lớp học điển hình có bao nhiêu sinh viên?",
        "Có sự khác biệt đáng kể về kích thước lớp học giữa các khóa học nhập môn và nâng cao không?",
        "Kích thước lớp học có khác nhau tùy theo ngành học không?",
        "Có những lớp học có kích thước nhỏ không?"
      ],
      "responses": [
        "Kích thước lớp học có thể khác nhau tùy theo khóa học và cấp độ học. Các khóa học nhập môn thường có kích thước lớp lớn hơn, trong khi các khóa học cấp cao thường nhỏ hơn, cho phép chú ý cá nhân hơn. Trung bình, kích thước lớp học có thể dao động từ khoảng 20 đến 100 sinh viên."
      ],
      "context_set": ""
    },
    {
      "tag": "study_abroad_programs",
      "patterns": [
        "Sinh viên có thể tham gia một học kỳ ở nước ngoài không?",
        "Trường có cung cấp các chương trình du học không?",
        "Có những cơ hội trao đổi quốc tế nào không?",
        "Có các chương trình cụ thể nào để du học không?",
        "Sinh viên có thể tham gia vào một chương trình du học như thế nào?"
      ],
      "responses": [
        "Có, trường cung cấp các chương trình du học cho phép sinh viên tham gia một học kỳ hoặc một năm học tại nhiều quốc gia khác nhau. Những chương trình này cung cấp cơ hội đặc biệt để hòa nhập văn hóa và làm giàu học thuật. Sinh viên có thể làm việc với văn phòng du học của trường để khám phá các chương trình có sẵn và quy trình đăng ký."
      ],
      "context_set": ""
    },
    {
      "tag": "financial_aid_international_students",
      "patterns": [
        "Trường có cung cấp học bổng hoặc hỗ trợ tài chính nào cho sinh viên quốc tế không?",
        "Có những lựa chọn hỗ trợ tài chính nào cho sinh viên quốc tế không?",
        "Có những học bổng nào dành cho sinh viên quốc tế?",
        "Có hỗ trợ tài chính nào cho những người không phải là công dân Mỹ không?",
        "Sinh viên quốc tế có thể nộp đơn xin hỗ trợ tài chính như thế nào?"
      ],
      "responses": [
        "Có, trường có thể cung cấp học bổng và hỗ trợ tài chính cho sinh viên quốc tế. Những cơ hội này được thiết kế để hỗ trợ việc học tập của những người không phải là công dân Mỹ và có thể khác nhau về tiêu chí đủ điều kiện. Sinh viên quốc tế có thể tìm hiểu về các học bổng và lựa chọn hỗ trợ tài chính có sẵn thông qua văn phòng hỗ trợ tài chính của trường."
      ],
      "context_set": ""
    },
    {
      "tag": "student_to_faculty_ratio",
      "patterns": [
        "Tỷ lệ sinh viên trên giảng viên trung bình tại trường đại học là bao nhiêu?",
        "Có bao nhiêu sinh viên thường được giao cho một giảng viên?",
        "Trường đại học có ưu tiên các lớp học nhỏ không?",
        "Các giáo sư có dễ tiếp cận ngoài giờ học không?"
      ],
      "responses": [
        "Tỷ lệ sinh viên trên giảng viên trung bình tại trường đại học thường là X:1, có nghĩa là có X sinh viên cho mỗi giảng viên. Trường đại học coi trọng giáo dục cá nhân hóa và cố gắng duy trì tỷ lệ sinh viên trên giảng viên thấp để thúc đẩy tương tác và hỗ trợ học thuật tốt hơn."
      ],
      "context_set": ""
    },
    {
      "tag": "student_organizations",
      "patterns": [
        "Có những loại tổ chức sinh viên nào để sinh viên tham gia?",
        "Có các câu lạc bộ hoặc tổ chức do sinh viên điều hành trong khuôn viên trường không?",
        "Sinh viên có thể tham gia vào các hoạt động ngoại khóa như thế nào?",
        "Những tổ chức sinh viên phổ biến nào tại trường đại học?",
        "Có cơ hội để giữ vai trò lãnh đạo trong các tổ chức sinh viên không?"
      ],
      "responses": [
        "Trường cung cấp một loạt các tổ chức sinh viên và câu lạc bộ phong phú phục vụ cho nhiều sở thích và đam mê khác nhau. Những tổ chức này cung cấp cơ hội để sinh viên tham gia vào các hoạt động ngoại khóa, phát triển kỹ năng lãnh đạo và xây dựng cảm giác cộng đồng. Sinh viên có thể tham gia các hội chợ câu lạc bộ, tham dự các buổi họp tổ chức và kết nối với các thành viên hiện tại để tham gia."
      ],
      "context_set": ""
    },
    {
      "tag": "recreational_facilities",
      "patterns": [
        "Các cơ sở giải trí trong khuôn viên trường như trung tâm thể dục hoặc sân thể thao như thế nào?",
        "Trường có phòng gym hoặc trung tâm thể dục cho sinh viên không?",
        "Có các cơ sở thể thao nào dành cho sinh viên không?",
        "Trường có cung cấp các môn thể thao nội bộ nào không?",
        "Sinh viên có thể sử dụng các cơ sở giải trí ngoài giờ hành chính không?"
      ],
      "responses": [
        "Trường cung cấp các cơ sở giải trí trong khuôn viên trường có thể bao gồm một phòng gym hiện đại, trung tâm thể dục, sân thể thao và các không gian khác cho nhiều hoạt động thể chất. Sinh viên có thể sử dụng các cơ sở này với thẻ sinh viên của mình, và một số trường cung cấp các môn thể thao nội bộ để sinh viên tham gia các cuộc thi thân thiện. Giờ hoạt động của các cơ sở này thường được thiết kế để phù hợp với lịch trình của sinh viên."
      ],
      "context_set": ""
    },
    {
      "tag": "admission_requirements_international_students",
      "patterns": [
        "Yêu cầu nhập học cho sinh viên quốc tế là gì?",
        "Có những yêu cầu bổ sung nào cho các ứng viên không phải là người Mỹ?",
        "Những bài kiểm tra tiêu chuẩn nào được chấp nhận cho sinh viên quốc tế?",
        "Có yêu cầu về trình độ tiếng Anh nào cho các ứng viên quốc tế không?",
        "Những tài liệu nào cần thiết cho đơn xin nhập học của sinh viên quốc tế?"
      ],
      "responses": [
        "Sinh viên quốc tế phải nộp đơn xin nhập học bao gồm bảng điểm học tập, điểm kiểm tra tiêu chuẩn (như SAT hoặc ACT), thư giới thiệu, bài tự luận cá nhân và bằng chứng về trình độ tiếng Anh (như điểm TOEFL hoặc IELTS). Mỗi trường có thể có các tiêu chí nhập học cụ thể, và sinh viên quốc tế tiềm năng có thể tham khảo trang web nhập học của trường để biết các yêu cầu chi tiết."
      ],
      "context_set": ""
    },
    {
      "tag": "experiential_learning_internships",
      "patterns": [
        "Hãy cho tôi biết về cách tiếp cận của trường đối với học tập trải nghiệm và thực tập.",
        "Trường khuyến khích sinh viên tích lũy kinh nghiệm thực tế như thế nào?",
        "Thực tập có phải là một phần phổ biến của chương trình học không?",
        "Trường có trung tâm dịch vụ nghề nghiệp để hỗ trợ sinh viên với các cơ hội thực tập không?",
        "Có những loại cơ hội học tập trải nghiệm nào có sẵn?"
      ],
      "responses": [
        "Trường nhấn mạnh học tập trải nghiệm và thực tập như những thành phần thiết yếu của giáo dục. Thông qua thực tập, sinh viên có thể tích lũy kinh nghiệm thực tế trong lĩnh vực học tập của họ và phát triển các kết nối nghề nghiệp quý giá. Trung tâm dịch vụ nghề nghiệp của trường đóng vai trò quan trọng trong việc giúp sinh viên đảm bảo các cơ hội thực tập và hợp tác. Ngoài ra, còn có thể có các cơ hội cho các dự án nghiên cứu, học tập phục vụ cộng đồng và tham gia vào các hoạt động cộng đồng."
      ],
      "context_set": ""
    },
    {
      "tag": "professor_accessibility",
      "patterns": [
        "Các giáo sư có dễ tiếp cận cho các giờ tư vấn và hỗ trợ học tập không?",
        "Các giáo sư có sẵn sàng ngoài giờ học để trả lời câu hỏi và thảo luận không?",
        "Các giáo sư có khuyến khích sinh viên tìm kiếm sự giúp đỡ hoặc làm rõ không?",
        "Sinh viên có thể lên lịch họp với giáo sư như thế nào?",
        "Có chính sách nào về việc giáo sư phản hồi email của sinh viên không?"
      ],
      "responses": [
        "Các giáo sư tại trường thường dễ tiếp cận và khuyến khích sinh viên tham dự các giờ tư vấn hoặc lên lịch gặp gỡ để nhận hỗ trợ học tập và thảo luận. Họ coi trọng việc giao tiếp mở với sinh viên và thường phản hồi email của sinh viên trong thời gian hợp lý. Xây dựng mối quan hệ mạnh mẽ với các giáo sư có thể cải thiện trải nghiệm học tập tổng thể."
      ],
      "context_set": ""
    },
    {
      "tag": "transportation_options",
      "patterns": [
        "Các lựa chọn giao thông đến và từ khuôn viên trường là gì?",
        "Trường có cung cấp dịch vụ vận chuyển không?",
        "Có các phương tiện giao thông công cộng gần khuôn viên trường không?",
        "Có dễ dàng di chuyển quanh khuôn viên trường và khu vực xung quanh mà không cần ô tô bằng cách sử dụng giao thông công cộng hoặc phương tiện nào khác không?",
        "Có các đường dẫn hoặc chương trình chia sẻ xe đạp không?"
      ],
      "responses": [
        "Trường có thể cung cấp dịch vụ xe đưa đón cho sinh viên đi lại đến và từ khuôn viên trường hoặc cung cấp dễ dàng tiếp cận các tùy chọn giao thông công cộng. Sinh viên có thể khám phá các hệ thống giao thông công cộng gần đó hoặc sử dụng các đường dẫn thân thiện với xe đạp để tiện lợi trong việc di chuyển. Ngoài ra, nhiều trường có khuôn viên dễ đi bộ, cho phép sinh viên di chuyển trong khu vực mà không cần ô tô."
      ],
      "context_set": ""
    }
  ]
}


In [38]:
# Function to perform synonym replacement
def synonym_replacement(tokens, limit):
    augmented_sentences = []
    for i in range(len(tokens)):
        synonyms = []
        for syn in wordnet.synsets(tokens[i]):
            for lemma in syn.lemmas():
                synonyms.append(lemma.name())
        if len(synonyms) > 0:
            num_augmentations = min(limit, len(synonyms))
            sampled_synonyms = random.sample(synonyms, num_augmentations)
            for synonym in sampled_synonyms:
                augmented_tokens = tokens[:i] + [synonym] + tokens[i+1:]
                augmented_sentences.append(' '.join(augmented_tokens))
    return augmented_sentences

In [39]:
# Function to read stopwords from a file
def load_stopwords(filepath):
    with open(filepath, 'r', encoding='utf-8') as file:
        return set(file.read().splitlines())

# Load Vietnamese stopwords
vietnamese_stopwords = load_stopwords('vietnamese-stopwords.txt')

# Initialize the lemmatizer
lemmatizer = WordNetLemmatizer()

text_data = []
labels = []
limit_per_tag = 40

# Assume intents is defined somewhere in your code
for intent in intents['intents']:
    augmented_sentences_per_tag = 0
    for example in intent['patterns']:
        tokens = nltk.word_tokenize(example.lower())
        filtered_tokens = [lemmatizer.lemmatize(token) for token in tokens if token not in vietnamese_stopwords and token.isalpha()]
        if filtered_tokens:
            text_data.append(' '.join(filtered_tokens))
            labels.append(intent['tag'])
            
            augmented_sentences = synonym_replacement(filtered_tokens, limit_per_tag - augmented_sentences_per_tag)
            for augmented_sentence in augmented_sentences:
                text_data.append(augmented_sentence)
                labels.append(intent['tag'])
                augmented_sentences_per_tag += 1
                if augmented_sentences_per_tag >= limit_per_tag:
                    break

print(len(text_data))
print(len(labels))

966
966


In [40]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(text_data)
y = labels

In [41]:
def find_best_model(X, y, test_size=0.2):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=100)


    models = [
        ('Logistic Regression', LogisticRegression(), {
            'penalty': ['l2'],
            'C': [0.1, 1.0, 10.0],
            'solver': ['liblinear'],
            'max_iter': [100, 1000, 10000]
        }),
        ('Multinomial Naive Bayes', MultinomialNB(), {'alpha': [0.1, 0.5, 1.0]}),
        ('Linear SVC', LinearSVC(), {
            'penalty': ['l2'],
            'loss': ['hinge', 'squared_hinge'],
            'C': [0.1, 1, 10],
            'max_iter': [100, 1000, 10000]
        }),
        ('Decision Tree', DecisionTreeClassifier(), {
            'max_depth': [5, 10, 20, None],
            'min_samples_split': [2, 5, 10],
            'min_samples_leaf': [1, 2, 4],
            'criterion': ['gini', 'entropy']
        }),
        ('Random Forest', RandomForestClassifier(), {
            'n_estimators': [100, 200, 300],
            'max_depth': [10, 20, None],
            'min_samples_split': [2, 5, 10],
            'min_samples_leaf': [1, 2, 4]
        })
    ]

    for name, model, param_grid in models:
        grid = GridSearchCV(model, param_grid, cv=3, n_jobs=-1)
        grid.fit(X_train, y_train)
        y_pred = grid.predict(X_test)
        score = accuracy_score(y_test, y_pred)
        print(f'{name}: {score:.4f} (best parameters: {grid.best_params_})')

    best_model = max(models, key=lambda x: GridSearchCV(x[1], x[2], cv=3, n_jobs=-1).fit(X_train, y_train).score(X_test, y_test))
    print(f'\nBest model: {best_model[0]}')

    # Fit the best model to the full training data
    best_model[1].fit(X, y)

    return best_model[1]

In [42]:
best_model = find_best_model(X, y)

Logistic Regression: 0.7216 (best parameters: {'C': 10.0, 'max_iter': 100, 'penalty': 'l2', 'solver': 'liblinear'})
Multinomial Naive Bayes: 0.5825 (best parameters: {'alpha': 0.1})
Linear SVC: 0.7268 (best parameters: {'C': 10, 'loss': 'squared_hinge', 'max_iter': 100, 'penalty': 'l2'})
Decision Tree: 0.5722 (best parameters: {'criterion': 'gini', 'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2})
Random Forest: 0.6701 (best parameters: {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 300})

Best model: Linear SVC


In [43]:
def chatbot_response(user_input):
    input_text = vectorizer.transform([user_input])
    predicted_intent = best_model.predict(input_text)[0]
    
    for intent in intents['intents']:
        if intent['tag'] == predicted_intent:
            response = random.choice(intent['responses'])
            break
            
    return response

In [44]:
print('Hello! I am a chatbot. How can I help you today? Type "quit" to exit.')
while True:
    user_input = input('> ')
    if user_input.lower() == 'quit':
        break
    response = chatbot_response(user_input)
    print(response)

Hello! I am a chatbot. How can I help you today? Type "quit" to exit.


In [45]:
import os
import pickle


if not os.path.exists('model'):
    os.makedirs('model')

if not os.path.exists('dataset'):
    os.makedirs('dataset')

# Save the trained model
with open('model/chatbot_model.pkl', 'wb') as f:
    pickle.dump(best_model, f)

# Save the vectorizer
with open('model/vectorizer.pkl', 'wb') as f:
    pickle.dump(vectorizer, f)

# Save the intents to the "dataset" folder
with open('dataset/intents.json', 'w') as f:
    json.dump(intents, f)